In [1]:
import pandas as pd
from pathlib import Path
from lxml import etree
import copy
import json
import re
import traceback as tb
from pkg_resources import resource_filename
import numpy as np
from packaging.version import Version
from datetime import datetime
from controle_coherence.utils import convert_xml_text, element_to_value_dict, get_duplicates, get_uniques, calc_seuil_interpolate
from controle_coherence.enum_report import msg_themes, msg_importance
from controle_coherence.assets_dpe import tv_table_to_value, complex_values_list, mutually_exclusive_elements, \
    elements_saisi, \
    expected_pt_liaison, default_isol_for_pt_isol_mais_inconnu, expected_components, bool_trad_fr, \
    modele_verification_paths, \
    ban_types, versions_dpe_cfg, \
    specific_values_list, id_generateur_pac_hybride, seuils_energie, seuils_energie_altitude, seuils_ges_altitude, \
    seuils_ges, zones_climatiques_altitude, get_current_valid_versions_dpe, \
    systeme_to_cle_repartition, materiau_mur_isolant, DPE_VERSION_ANTERIEUR, get_latest_dpe_version, DATE_APPLICATION_PETITE_SURFACE, get_datetime_now, seuils_tertiaire, DATE_APPLICATION_BLOCAGE_CONTROLE_RCU

from controle_coherence.assets_audit import versions_audit_cfg, AUDIT_VERSION_ANTERIEUR, \
    get_current_valid_versions_audit

/tmp/ipykernel_10583/206342416.py:8: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import lxml 
from lxml import etree
from lxml.etree import ElementTree,Element,SubElement
import re
import copy

In [3]:
from controle_coherence.controle_coherence import EngineDPE,EngineAudit,ReportDPE
parser = etree.XMLParser(remove_blank_text=True)


In [4]:
self = EngineDPE()
engine = self
report = ReportDPE()
tv_reg = self.valeur_table['rendement_regulation']
tv_em = self.valeur_table['rendement_emission']
f = str((engine.mdd_path / 'exemples_metier' / 'cas_test_immeuble_1_valid.xml'))
dpe = etree.parse(f, parser)

In [61]:
logement = dpe.find('logement')

for el in logement.iterfind('*//mur//enum_type_isolation_id'):
    el.text='2'

In [62]:
for el_paroi_lourde in logement.iterfind('*//mur//paroi_lourde'):
    materiau = int(el_paroi_lourde.getparent().find('enum_materiaux_structure_mur_id').text)
    paroi_lourde = int(el_paroi_lourde.text)
    expected_paroi_lourde=self.enum_table['materiaux_structure_mur'].loc[materiau].fillna(0).paroi_lourde
    min_epaisseur =self.enum_table['materiaux_structure_mur'].loc[materiau].fillna(0).paroi_lourde_epaisseur
    type_isolation =  int(el_paroi_lourde.getparent().find('enum_type_isolation_id').text)
    type_adjacence =  int(el_paroi_lourde.getparent().find('enum_type_adjacence_id').text)
    if el_paroi_lourde.getparent().find('epaisseur_structure') is not None:
        
        ep =  int(el_paroi_lourde.getparent().find('epaisseur_structure').text)
    else:
        ep = 0
    
    if (expected_paroi_lourde==1) and (type_isolation in (2,4)) and (ep>=min_epaisseur) and (paroi_lourde==0) and (type_adjacence==1):
        epaisseur_txt = ep
        if epaisseur_txt is None:
            epaisseur_txt='non définie'
        
        msg=f"""
un mur en {self.display_enum_traduction('enum_materiaux_structure_mur_id',materiau)} devrait être déclaré comme paroi lourde mais ne l'est pas
matériau : {self.display_enum_traduction('enum_materiaux_structure_mur_id',materiau)}
épaisseur : {epaisseur_txt}
type_isolation : {self.display_enum_traduction('enum_type_isolation_id',type_isolation)} 
donnant sur :{self.display_enum_traduction('enum_type_adjacence_id',type_adjacence)}
        """

In [65]:
print(msg)


un mur en {9: "Murs en briques pleines doubles avec lame d'air"} devrait être déclaré comme paroi lourde mais ne l'est pas
matériau : {9: "Murs en briques pleines doubles avec lame d'air"}
épaisseur : 20
type_isolation : {2: 'Non isolé'} 
donnant sur :{1: 'Extérieur'}
        


In [59]:
min_epaisseur

12.0

In [57]:
(type_isolation in (2,4))

False

In [58]:
type_isolation

3

In [55]:
print(msg)


un mur en {8: 'Murs en briques pleines simples'} devrait être déclaré comme paroi lourde mais ne l'est pas
matériau : {8: 'Murs en briques pleines simples'}
épaisseur : 19
type_isolation : {2: 'Non isolé'} 
        


In [41]:
None>12

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [39]:
expected_paroi_lourde

1.0

In [40]:
msg

't'

In [15]:
min_epaisseur

12.0

In [106]:
type_liaison_num = [int(el.text) for el in
                            list(logement.iterfind('*//pont_thermique/donnee_entree/enum_type_liaison_id'))]

In [110]:
True and (not {5}.issubset(type_liaison_num))

False

In [61]:
calcul_pont_thermique_baie = [(self.enum_table['type_vitrage'].loc[int(el.text)].calcul_pont_thermique_baie, int(el.getparent().find('enum_type_adjacence_id').text), int(el.getparent().find('enum_inclinaison_vitrage_id').text),el) for
                              el in logement.iterfind('*//baie_vitree//enum_type_vitrage_id')]

el_baie_ext_pt = [el[3] for el in calcul_pont_thermique_baie if el[0:3]==(1, 1, 3)]

calcul_pont_thermique_baie = [el[0:3] for el in calcul_pont_thermique_baie]
# on a au moins une baie extérieure verticale
baie_ext_pt = {(1, 1, 3)}.issubset(set(calcul_pont_thermique_baie))

calcul_pont_thermique_mur = [(self.enum_table['materiaux_structure_mur'].loc[int(el.text)].calcul_pont_thermique_baie, int(el.getparent().find('enum_type_adjacence_id').text),el) for
                             el in logement.iterfind('*//mur//enum_materiaux_structure_mur_id')]

el_mur_ext_pt = [el[2] for el in calcul_pont_thermique_mur if el[0:2] == (1, 1)]

calcul_pont_thermique_mur = [el[0:2] for el in calcul_pont_thermique_mur]
# on a au moins un mur extérieur et on a pas de murs en bois
mur_ext_pt = {(1, 1)}.issubset(set(calcul_pont_thermique_mur)) and (not {(1, 0)}.issubset(set(calcul_pont_thermique_mur)))

In [ ]:
paroi_lourde_pb = [(int(el.text),el) for
                              el in logement.iterfind('*//plancher_bas//paroi_lourde')]



el_paroi_lourde_pb =[el[1] for el in paroi_lourde_pb if el[0]==1]
# on a que des plancher bas lourds
paroi_lourde_pb = pd.Series([el[0] for el in paroi_lourde_pb]).min()==1

paroi_lourde_ph = [(int(el.text),el) for
                              el in logement.iterfind('*//plancher_haut//paroi_lourde')]



el_paroi_lourde_ph =[el[1] for el in paroi_lourde_ph if el[0]==1]
# on a que des plancher bas lourds
paroi_lourde_ph = pd.Series([el[0] for el in paroi_lourde_ph]).min()==1

paroi_lourde_mur_ext = [(int(el.text),int(el.getparent().find('enum_type_adjacence_id').text),el) for
                              el in logement.iterfind('*//mur//paroi_lourde')]

paroi_lourde_mur_ext=[el for el in paroi_lourde_mur_ext if el[1]==1]
el_paroi_lourde_mur =[el[1] for el in paroi_lourde_mur_ext if el[0]==1]
# on a que des murs lourds
paroi_lourde_mur_ext = pd.Series([el[0] for el in paroi_lourde_mur_ext]).min()==1

False

In [89]:
paroi_lourde_mur

False

In [63]:
el_baie_ext_pt

[<Element enum_type_vitrage_id at 0x7f9b0c364b00>,
 <Element enum_type_vitrage_id at 0x7f9b0c364a00>]

In [62]:
el_mur_ext_pt

[<Element enum_materiaux_structure_mur_id at 0x7f9b0c20af40>]

In [59]:
calcul_pont_thermique_baie[0][0:3]

(1, 1, 3)

In [56]:
mur_ext_pt

True

In [57]:
baie_ext_pt

True

In [38]:
calcul_pont_thermique_mur

[(1, 1), (1, 14), (1, 13), (1, 13)]

In [33]:
calcul_pont_thermique

[(1, 1), (1, 1)]

In [25]:
type_pose_baie_vitree

[1, 1]

In [ ]:
parser = etree.XMLParser(remove_blank_text=True)
f = str((engine.mdd_path / 'exemples_metier' / 'cas_anomalie_veilleuse.xml'))
dpe = etree.parse(f, parser)
report = ReportDPE()

logement = dpe.find('logement')

In [54]:
tv_generateur_combustion=engine.valeur_table['generateur_combustion']
gen_veilleuse = tv_generateur_combustion.loc[tv_generateur_combustion.pveil>0]

enum_type_generateur_ch_id_avec_pveilleuse=gen_veilleuse.explode('enum_type_generateur_ch_id').enum_type_generateur_ch_id.dropna().astype(str).unique()
enum_type_generateur_ecs_id_avec_pveilleuse=gen_veilleuse.explode('enum_type_generateur_ecs_id').enum_type_generateur_ecs_id.dropna().astype(str).unique()

el_type_generateur_ch = list(logement.iterfind('*//enum_type_generateur_ch_id'))
el_type_generateur_ecs = list(logement.iterfind('*//enum_type_generateur_ecs_id'))

In [58]:
el_type_generateur_ecs[0].text

'71'

In [59]:
enum_type_generateur_ecs_id_avec_pveilleuse

array(['45', '92', '46', '93', '47', '94', '48', '95', '51', '98', '54',
       '101', '55', '102', '58', '105', '59', '106', '60', '107', '63',
       '110', '64', '111', '65', '112', '66', '113'], dtype=object)

In [49]:
self =engine

In [60]:
for el in el_type_generateur_ch:
    
    if el.text in enum_type_generateur_ch_id_avec_pveilleuse:
        generateur = el.getparent().getparent()
        enum_methode_saisie_carac_sys_id = generateur.find('*//enum_methode_saisie_carac_sys_id')
        if generateur.find('*//pveilleuse') is None and enum_methode_saisie_carac_sys_id.text=='1':
            msg = f"""
le champ pveilleuse doit être renseigné pour les générateurs :
{self.display_enum_traduction("enum_type_generateur_ch_id", int(el.text))}
si la saisie est par défaut {self.display_enum_traduction("enum_methode_saisie_carac_sys_id", int(enum_methode_saisie_carac_sys_id.text))}
"""
            related_objects = [generateur, el]
            report.generate_msg(msg, msg_type='erreur_logiciel',
                                msg_theme='missing_required_element',
                                related_objects=related_objects,
                                msg_importance='blocker_as_warning')
            
for el in el_type_generateur_ecs:
    
    if el.text in enum_type_generateur_ecs_id_avec_pveilleuse:
        generateur = el.getparent().getparent()
        enum_methode_saisie_carac_sys_id = generateur.find('*//enum_methode_saisie_carac_sys_id')
        if generateur.find('*//pveilleuse') is None and enum_methode_saisie_carac_sys_id.text=='1':
            msg = f"""
le champ pveilleuse doit être renseigné pour les générateurs :
{self.display_enum_traduction("enum_type_generateur_ecs_id", int(el.text))}
si la saisie est par défaut {self.display_enum_traduction("enum_methode_saisie_carac_sys_id", int(enum_methode_saisie_carac_sys_id.text))}
"""
            related_objects = [generateur, el]
            report.generate_msg(msg, msg_type='erreur_logiciel',
                                msg_theme='missing_required_element',
                                related_objects=related_objects,
                                msg_importance='blocker_as_warning')

In [51]:
report.error_software

[{'message': "\nle champ pveilleuse doit être renseigné pour les générateurs :\n{86: 'Chaudière gaz classique 1981-1985'}\nsi la saisie est par défaut {1: 'valeurs forfaitaires'}\n",
  'thematique': 'missing_required_element',
  'objets_concerne': [<Element generateur_chauffage at 0x243f3631a00>,
   <Element enum_type_generateur_ch_id at 0x243f0dfd040>],
  'importance': 'blocker'},
 {'message': "\nle champ pveilleuse doit être renseigné pour les générateurs :\n{86: 'Chaudière gaz classique 1981-1985'}\nsi la saisie est par défaut {1: 'valeurs forfaitaires'}\n",
  'thematique': 'missing_required_element',
  'objets_concerne': [<Element generateur_chauffage at 0x243f3631a00>,
   <Element enum_type_generateur_ch_id at 0x243f0dfd040>],
  'importance': 'blocker'}]

In [62]:
from controle_coherence.enum_report import msg_importance

In [63]:
report = ReportDPE()
engine.controle_coherence_enveloppe(dpe.find('logement'), report)

In [64]:
self=report
xml_reg = dpe

In [65]:
self.report

{'validation_xsd': {},
 'warning_logiciel': [],
 'warning_saisie': [{'message': "\n    pour le calcul du b d'une paroi de type mur celle ci est considérée isolé : Oui\n    alors que l'isolation de la paroi de type mur est {2: 'Non isolé'}\n    Cet avertissement peut être ignoré pour le cas particulier où l'isolation des parois donnant sur le local non chauffé est inhomogène et le type d'isolation de cette paroi est minoritaire.\n    ",
   'thematique': 'incoherence_enveloppe',
   'objets_concerne': [<Element enum_type_isolation_id at 0x26452f04d40>],
   'importance': 'major'},
  {'message': "\n    pour le calcul du b d'une paroi de type mur celle ci est considérée isolé : Oui\n    alors que l'isolation de la paroi de type mur est {2: 'Non isolé'}\n    Cet avertissement peut être ignoré pour le cas particulier où l'isolation des parois donnant sur le local non chauffé est inhomogène et le type d'isolation de cette paroi est minoritaire.\n    ",
   'thematique': 'incoherence_enveloppe',


In [81]:
engine = EngineDPE()

parser = etree.XMLParser(remove_blank_text=True)
f = str((engine.mdd_path / 'exemples_metier' / 'cas_test_appt_1.xml'))

report = ReportDPE()
dpe = etree.parse(f, parser)
dpe.find('*//pont_thermique/donnee_entree/tv_pont_thermique_id').text = "22"
engine.controle_coherence_pont_thermique(dpe.find('logement'), report)

In [82]:
report = ReportDPE()
dpe = etree.parse(f, parser)
for el in dpe.iterfind('*//pont_thermique/donnee_entree/tv_pont_thermique_id'):
    id_ = int(el.text)
    id_ += 5
    el.text = str(id_)
engine.controle_coherence_pont_thermique(dpe.find('logement'), report)


In [85]:
report = ReportDPE()
dpe = etree.parse(f, parser)
dpe.find('*//pont_thermique/donnee_entree/tv_pont_thermique_id').text = "22"
engine.controle_coherence_pont_thermique(dpe.find('logement'), report)
assert (len(report.warning_input) == 2)
report = ReportDPE()
dpe.find('*//pont_thermique/donnee_entree/tv_pont_thermique_id').text = "42"
engine.controle_coherence_pont_thermique(dpe.find('logement'), report)
assert (len(report.warning_input) == 2)

In [86]:
report.warning_input

[{'message': "\n    le type d'isolation des murs saisi dans les ponts thermiques est : ITI\n    ceci est incohérent avec le type d'isolation saisi pour les murs du logement : Non isolé\n    ",
  'thematique': 'incoherence_enveloppe',
  'objets_concerne': [<Element tv_pont_thermique_id at 0x264525f5480>],
  'importance': 'major'},
 {'message': "\n    il existe des ponts thermiques de type plancher haut / Mur mais aucun plancher haut n'est déclaré dans le modèle de donnée. \n    ",
  'thematique': 'anomaly_pont_thermique',
  'objets_concerne': [<Element tv_pont_thermique_id at 0x264525f5480>],
  'importance': 'major'}]

In [83]:
len(report.warning_input)

1

In [84]:
report.warning_input

[{'message': "\n    le type d'isolation des murs saisi dans les ponts thermiques est : ITI ou ITI+ITR\n    ceci est incohérent avec le type d'isolation saisi pour les murs du logement : Non isolé\n    ",
  'thematique': 'incoherence_enveloppe',
  'objets_concerne': [<Element tv_pont_thermique_id at 0x26452f30f80>,
   <Element tv_pont_thermique_id at 0x26452f304c0>,
   <Element tv_pont_thermique_id at 0x26452f30380>,
   <Element tv_pont_thermique_id at 0x26452f30880>,
   <Element tv_pont_thermique_id at 0x26452d5a940>],
  'importance': 'major'}]

In [76]:
report.report

{'validation_xsd': {},
 'warning_logiciel': [],
 'warning_saisie': [{'message': "\n    le type d'isolation des murs saisi dans les ponts thermiques est : ITI+ITR\n    ceci est incohérent avec le type d'isolation saisi pour les murs du logement : ITI ou Non isolé\n    ",
   'thematique': 'incoherence_enveloppe',
   'objets_concerne': [<Element tv_pont_thermique_id at 0x26452f30a40>],
   'importance': 'major'},
  {'message': "\n    le type d'isolation des planchers bas saisi dans les ponts thermiques est : ITI\n    ceci est incohérent avec le type d'isolation saisi pour les planchers bas du logement : ITE\n    ",
   'thematique': 'anomaly_pont_thermique',
   'objets_concerne': [<Element tv_pont_thermique_id at 0x26452f30d80>,
    <Element tv_pont_thermique_id at 0x26452c639c0>],
   'importance': 'major'},
  {'message': "\n    le type d'isolation des planchers hauts saisi dans les ponts thermiques est : ITI\n    ceci est incohérent avec le type d'isolation saisi pour les planchers hauts d

In [68]:
dpe = etree.parse(f, parser)
dpe.find('*//pont_thermique/donnee_entree/tv_pont_thermique_id').text = "22"
engine.controle_coherence_pont_thermique(dpe.find('logement'), report)
assert (len(report.warning_input) == 2)

AssertionError: 

In [66]:
msg_importance_for_report = copy.deepcopy(msg_importance)
msg_importance_for_report = self.manage_blocker_as_warning(xml_reg, engine, msg_importance_for_report)


In [67]:
self.report

{'validation_xsd': {},
 'warning_logiciel': [],
 'warning_saisie': [],
 'erreur_logiciel': [],
 'erreur_saisie': [],
 'notification': [],
 'message_principal': ''}

In [57]:
msg_importance_for_report = copy.deepcopy(msg_importance)
msg_importance_for_report = self.manage_blocker_as_warning(xml_reg, engine, msg_importance_for_report)
self.enrich_issues_message_and_meta_data(xml_reg, msg_importance_for_report,engine)
if len(self.report['erreur_saisie']) + len(self.report['erreur_logiciel']) > 0:
    self.report[
        'message_principal'] += f" IL EXISTE DES ERREURS BLOQUANTES EMPECHANT LA SOUMISSION {engine.DENOMINATION_OBJET_XML_REG} A L'OBSERVATOIRE."


In [58]:
report.generate_report(dpe,engine)

{'validation_xsd': {},
 'warning_logiciel': [],
 'warning_saisie': [],
 'erreur_logiciel': [],
 'erreur_saisie': [],
 'notification': [],
 'message_principal': ''}

In [32]:
report.report

{'validation_xsd': {},
 'warning_logiciel': [],
 'warning_saisie': [{'message': "\n    pour le calcul du b d'une paroi de type mur celle ci est considérée isolé : Oui\n    alors que l'isolation de la paroi de type mur est {2: 'Non isolé'}\n    Cet avertissement peut être ignoré pour le cas particulier où l'isolation des parois donnant sur le local non chauffé est inhomogène et le type d'isolation de cette paroi est minoritaire.\n    ",
   'thematique': 'incoherence_enveloppe',
   'objets_concerne': [<Element enum_type_isolation_id at 0x264525c8740>],
   'importance': 'major'},
  {'message': "\n    pour le calcul du b d'une paroi de type mur celle ci est considérée isolé : Oui\n    alors que l'isolation de la paroi de type mur est {2: 'Non isolé'}\n    Cet avertissement peut être ignoré pour le cas particulier où l'isolation des parois donnant sur le local non chauffé est inhomogène et le type d'isolation de cette paroi est minoritaire.\n    ",
   'thematique': 'incoherence_enveloppe',


In [25]:
len(report.report['warning_saisie'])

4

NameError: name 'EngineAudit' is not defined

In [1]:
import openpyxl

In [2]:
path = Path("..") / "modele_donnee"

export_excel_base_applicative_path =  path/'export_excel_base_applicative'
export_excel_base_applicative_path.mkdir(exist_ok=True,parents=True)

enum_tables = openpyxl.load_workbook(path /'enum_tables.xlsx')

In [10]:
import pandas as pd
import json
import os
import copy
import yaml
import numpy as np
from pathlib import Path
from lxml.etree import SubElement
from lxml.etree import ElementTree, XMLSchema, Element
from lxml import etree

WRITE_IN_PLACE = True

## load excel data


path = Path("..") / "modele_donnee"

df_mdd = pd.read_excel(path / 'modele_donnee.xlsx')
df_mdd = df_mdd[['description', 'nom_variable', 'parent_xpath', 'type', 'condition', 'DPE NEUF']]

enum_table = pd.read_excel(path / 'enum_tables.xlsx', sheet_name=None, dtype=str)
print(len(enum_table))
enum_table_audit = pd.read_excel(path / 'enum_tables_audit.xlsx', sheet_name=None, dtype=str)

enum_table_audit['version_audit']['id']=(enum_table_audit['version_audit'].id.astype(float).round(1)).astype(str)

enum_doc_audit = pd.read_excel(path / 'enum_doc_audit.xlsx', sheet_name='doc', dtype=str)

valeur_table = pd.read_excel(path / 'valeur_tables.xlsx', sheet_name=None)
valeur_table['coef_masque_lointain_non_homogene'] = valeur_table['coef_masque_lointain_non_homoge']
valeur_table['reseau_chaleur'] = valeur_table['reseau_chaleur_2020']
del valeur_table['reseau_chaleur_2020']
del valeur_table['reseau_chaleur_2021']
del valeur_table['coef_masque_lointain_non_homoge']

enums = [el for el in df_mdd.nom_variable.dropna().values.tolist() if el.startswith('enum')]
enums_without_id = [el for el in enums if not el.endswith('id')]

df_mdd['parent_xpath_neuf'] = df_mdd['parent_xpath'].str.replace('dpe.logement.', 'dpe.logement_neuf.')

df_mdd['condition'] = (df_mdd.condition == 'requis').replace({True: 'requis', False: 'optionnel'})

null = (df_mdd['DPE NEUF'] == 'non demandé') | (df_mdd['DPE NEUF'].isnull())
df_mdd.loc[null, 'parent_xpath_neuf'] = np.nan

mdd = df_mdd.dropna(subset=['nom_variable']).to_dict(orient='records')

mdd_neuf = [el.copy() for el in mdd if el['parent_xpath_neuf'] is not np.nan]

for el in mdd_neuf:
    el['parent_xpath'] = el['parent_xpath_neuf']
    condition = el['DPE NEUF']
    if condition != "requis":
        condition = 'optionnel'
    el['condition'] = condition

mdd.extend(mdd_neuf)

# suppresion de la ref dpe/audit pour logement et administratif (qui est maintenant une ref.)
for el in mdd:
    el['parent_xpath'] = el['parent_xpath'].replace('dpe.logement.', 'logement.')
    el['parent_xpath'] = el['parent_xpath'].replace('dpe.administratif', 'administratif')

## write enums dict


enums_dict = {k: v.astype(str).set_index('id').lib.str.strip().str.lower().to_dict() for k, v in enum_table.items() if 'lib' in v and 'id' in v}
enums_dict['classe_etiquette'] = {k: v.upper() for k, v in enums_dict['classe_etiquette'].items()}
for k, v in enums_dict.items():
    for k1, v1 in v.items():
        if isinstance(v1, str):
            v[k1] = v1.replace('\xa0', ' ')

with open(path / 'enums.json', 'w', encoding='utf-8') as f:
    json.dump(enums_dict, f, indent=4, ensure_ascii=False)

enums_audit_doc = enum_doc_audit.set_index('nom enum').to_dict()['documentation']

enums_audit_dict = {k: v[~v.lib.isna()].astype(str).set_index('id').lib.str.strip().str.lower().to_dict() for k, v in enum_table_audit.items() if 'lib' in v and 'id' in v}

# enregistrement des enums audit dans un .json
for enum_name, id_lib in enums_audit_dict.items():
    for _id, lib in id_lib.items():
        if isinstance(lib, str):
            id_lib[_id] = lib.replace('\xa0', ' ')

with open(path / 'enums_audit.json', 'w', encoding='utf-8') as f:
    json.dump(enums_audit_dict, f, indent=4, ensure_ascii=False)

##  load xsd

# ## load xsd

from lxml.etree import ElementTree, XMLSchema
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True)
path_xsd = r'..\modele_donnee\modele_commun_DPE_audit_reg.xsd'
# path_xsd = r'.\DPE_dev.xsd'
schema = XMLSchema(file=path_xsd)

parser = etree.XMLParser(remove_blank_text=True)

et = etree.parse(path_xsd, parser)
root = et.getroot()

xs = '{http://www.w3.org/2001/XMLSchema}'
obsdpe = '{https://gitlab.com/observatoire-dpe/observatoire-dpe/-/tree/master/modele_donnee}'

namespaces = {'xs': 'http://www.w3.org/2001/XMLSchema',
              'obsdpe': "https://gitlab.com/observatoire-dpe/observatoire-dpe/-/tree/master/modele_donnee"}

all_el = list(root.iterfind('*//xs:element', namespaces=namespaces))

all_doc = list(root.iterfind('*//xs:documentation', namespaces=namespaces))

## xsd elements ->  object model (dict)

list_keys = list()
list_var = list()
for i, el in enumerate(all_el):
    #     if el.attrib.get('name')=='annee_construction':
    #         qdsqds
    caracs = dict(el.attrib)
    caracs['self'] = el
    if 'name' in caracs:
        is_complex = len([child for child in el.getchildren() if 'complexType' in str(child.tag)]) > 0
        if not is_complex:
            doc = el.find(f'xs:annotation/xs:documentation', namespaces=namespaces)
            if doc is not None:
                caracs.update({'documentation': doc.text})
            appinfo = el.find(f'xs:annotation/xs:appinfo', namespaces=namespaces)
            if appinfo is not None:
                caracs.update(appinfo.attrib)
                if appinfo.text is not None:
                    caracs.update({'appinfo': appinfo.text})
            restriction = el.find(f'xs:simpleType/xs:restriction', namespaces=namespaces)
            if restriction is not None:
                caracs.update(restriction.attrib)
                if len(restriction.getchildren()) >= 2 and restriction.attrib.get('base') == 'xs:int':
                    caracs.update({'len_enum': np.max([int(el.attrib['value']) for el in restriction.getchildren() if not 'pattern' in str(el.tag)])})
            list_var.append(caracs)
for el in list_var:
    if el.get('minOccurs') == '0':
        el['condition'] = 'optionnel'
    else:
        el['condition'] = 'requis'

dpe_neuf = [el for el in list_var if 'logement_neuf' in el.get('source', '')]

for el in list_var:
    el['xpath'] = el.get('source', '')

for el in mdd:
    el['xpath'] = el['parent_xpath'].replace('.', '/') + '/' + el["nom_variable"]

## update version

# # load and update version

import yaml

with open('../versions.yml', 'r',encoding='utf-8') as f:
    versions = yaml.safe_load(f)

v_audit = versions['model_audit']
v_dpe = versions['model_dpe']
v_commun = versions['modele_commun']

model_version_commun = f"Version {v_commun['version']} - {v_commun['date']} : {v_commun['titre']}"

model_version_audit = f"Version V{v_audit['version']} - {v_audit['date']} : {v_audit['titre']}"

model_version_dpe = f"Version V{v_dpe['version']} - {v_dpe['date']} : {v_dpe['titre']}"

dpe = root.xpath(f'//xs:element[@name="dpe"]', namespaces=namespaces)[0]
doc = dpe.xpath('xs:annotation/xs:documentation', namespaces=namespaces)[0]
doc.text = model_version_dpe


audit = root.xpath(f'//xs:element[@name="audit"]', namespaces=namespaces)[0]

documentation = audit.find('xs:annotation/xs:documentation', namespaces=namespaces)
documentation.text = f'{model_version_audit} ////// \n\t En cohérence avec la version dpe : {model_version_dpe}'

documentation = root.find('xs:annotation/xs:documentation', namespaces=namespaces)
documentation.text = f'{model_version_commun} ////// \n\t  {model_version_dpe} ////// \n\t  {model_version_audit}'

## process doc

### reformat doc

all_doc = list(root.iterfind(f'*//xs:documentation', namespaces=namespaces))

for doc in all_doc:
    txt = doc.text.replace('\n', '').strip()
    txt = ' '.join(txt.split())
    doc.text = txt

et.write('test.xsd', pretty_print=True, xml_declaration=True, encoding='utf-8')

all_el = list(root.iterfind('*//xs:element', namespaces=namespaces))


77


C:\Users\breitwiller\AppData\Local\Temp\ipykernel_9916\4115259542.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df_mdd['parent_xpath_neuf'] = df_mdd['parent_xpath'].str.replace('dpe.logement.', 'dpe.logement_neuf.')


In [13]:
for el in all_el:
    pass

In [17]:
enums_xsd = [el for el in list_var if el['name'].startswith('enum_')]


In [21]:
el=enums_xsd[0]['self']

In [61]:
path_dpe_complet = path / 'DPE_complet.xsd'


In [62]:
path_audit_reg = path / 'audit_reg.xsd'


In [86]:
path = Path("..") / "modele_donnee"





In [54]:
to_del

['version',
 'modele_dpe',
 'usage_fonctionnel_batiment',
 'categorie_erp',
 'methode_application_dpe_log',
 'methode_application_dpe_ter',
 'periode_construction',
 'periode_isolation',
 'zone_climatique',
 'classe_altitude',
 'methode_saisie_u',
 'methode_saisie_u0',
 'methode_saisie_uporte',
 'methode_saisie_pont_thermique',
 'methode_saisie_perf_vitrage',
 'methode_saisie_carac_sys',
 'methode_saisie_q4pa_conv',
 'methode_saisie_fact_couv_sol',
 'classe_inertie',
 'type_adjacence',
 'cfg_isolation_lnc',
 'materiaux_structure_mur',
 'type_isolation',
 'type_doublage',
 'type_plancher_bas',
 'type_plancher_haut',
 'type_vitrage',
 'inclinaison_vitrage',
 'type_materiaux_menuiserie',
 'type_baie',
 'type_pose',
 'type_gaz_lame',
 'orientation',
 'type_fermeture',
 'type_porte',
 'type_liaison',
 'type_ventilation',
 'periode_installation_fr',
 'type_generateur_fr',
 'type_installation_solaire',
 'type_installation',
 'methode_calcul_conso',
 'usage_generateur',
 'cfg_installation_ch',

In [53]:
enum_tables_only_dpe.sheetnames

['index']

In [22]:
el.attrib

{'name': 'enum_usage_fonctionnel_batiment_id'}

In [9]:
enum_tables.get_sheet_names()

C:\Users\breitwiller\AppData\Local\Temp\ipykernel_9916\598496956.py:1: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  enum_tables.get_sheet_names()


['index',
 'version',
 'modele_dpe',
 'usage_fonctionnel_batiment',
 'categorie_erp',
 'methode_application_dpe_log',
 'methode_application_dpe_ter',
 'periode_construction',
 'periode_isolation',
 'zone_climatique',
 'classe_altitude',
 'methode_saisie_u',
 'methode_saisie_u0',
 'methode_saisie_uporte',
 'methode_saisie_pont_thermique',
 'methode_saisie_perf_vitrage',
 'methode_saisie_carac_sys',
 'methode_saisie_q4pa_conv',
 'methode_saisie_fact_couv_sol',
 'classe_inertie',
 'type_adjacence',
 'cfg_isolation_lnc',
 'materiaux_structure_mur',
 'type_isolation',
 'type_doublage',
 'type_plancher_bas',
 'type_plancher_haut',
 'type_vitrage',
 'inclinaison_vitrage',
 'type_materiaux_menuiserie',
 'type_baie',
 'type_pose',
 'type_gaz_lame',
 'orientation',
 'type_fermeture',
 'type_porte',
 'type_liaison',
 'type_ventilation',
 'periode_installation_fr',
 'type_generateur_fr',
 'type_installation_solaire',
 'type_installation',
 'methode_calcul_conso',
 'usage_generateur',
 'cfg_install

In [4]:
engine = EngineDPE()

parser = etree.XMLParser(remove_blank_text=True)
f = str((engine.mdd_path / 'exemples_metier' / 'cas_test_immeuble_1_valid_generation_cascade.xml'))
dpe = etree.parse(f, parser)
report = engine.run_controle_coherence(dpe)
nb_errors = len(report['erreur_logiciel']) + len(report['erreur_saisie'])
assert (report['validation_xsd']['valid'] == True)
assert (nb_errors == 0)
L_warning = len(report['warning_logiciel'])+len(report['warning_saisie'])
for el in dpe.iterfind('*//priorite_generateur_cascade'):
    el.text= '1'
report = engine.run_controle_coherence(dpe)
nb_errors = len(report['erreur_logiciel']) + len(report['erreur_saisie'])
assert (report['validation_xsd']['valid'] == True)
assert (nb_errors == 0)
L_warning_sans_priorite = len(report['warning_logiciel']) + len(report['warning_saisie'])

assert(L_warning_sans_priorite==L_warning)

AssertionError: 

In [5]:
report['erreur_logiciel']

[{'message': "\n    le champ rendement_generation vaut 0 pour un générateur de chauffage de type {85: 'Chaudière gaz classique avant 1981'}\n    Ceci n'est autorisé que dans les cas suivants:\n    - générateur à combustion en cascade avec priorité qui est non prioritaire \n    - un appoint dans une installation collective base + appoint.\n    - une installation de chauffage avec chauffage solaire dont le fch est de 1 (couverture du besoin à 100% grâce à l'installation solaire)\n    Le générateur en question ne correspond pas à cette configuration.\n    ce générateur est prioritaire. ordre de priorité : 1.",
  'thematique': "une donnée de sortie est saisie à 0 pour un générateur qui n'est pas en cascade avec priorité et non prioritaire.",
  'objets_concerne': ['/dpe/logement/installation_chauffage_collection/installation_chauffage/generateur_chauffage_collection/generateur_chauffage[2]',
   '/dpe/logement/installation_chauffage_collection/installation_chauffage/generateur_chauffage_coll

In [ ]:
from controle_coherence.controle_coherence import EngineDPE


In [4]:
#import requests
#
#r=requests.post('http://localhost:5000/controle_coherence',data=etree.tostring(dpe))

In [ ]:
from controle_coherence.controle_coherence import EngineDPE

In [ ]:
engine = EngineDPE()

In [ ]:
engine.var_req_dict.keys()

In [ ]:
mdd_path = Path("..")/"modele_donnee"

xml_validator = etree.XMLSchema(file=str((mdd_path/"audit_reg.xsd").absolute()))


In [4]:
engine = EngineDPE()

In [7]:
engine.var_req_dict.keys()

dict_keys(['enum_methode_application_dpe_log_id', 'enum_methode_application_dpe_ter_id', 'enum_methode_saisie_u_id', 'enum_methode_saisie_u0_id', 'enum_methode_saisie_uporte_id', 'enum_methode_saisie_pont_thermique_id', 'enum_methode_saisie_perf_vitrage_id', 'enum_methode_saisie_carac_sys_id', 'enum_methode_saisie_q4pa_conv_id', 'enum_methode_saisie_fact_couv_sol_id', 'enum_type_adjacence_id', 'enum_type_vitrage_id', 'enum_type_fermeture_id', 'enum_type_installation_solaire_id', 'enum_type_installation_id', 'enum_methode_calcul_conso_id', 'enum_cfg_installation_ch_id', 'enum_cfg_installation_ecs_id', 'enum_type_generateur_ch_id', 'enum_type_generateur_ecs_id', 'enum_statut_geocodage_ban_id'])

In [5]:
mdd_path = Path("..")/"modele_donnee"

xml_validator = etree.XMLSchema(file=str((mdd_path/"audit_reg.xsd").absolute()))


In [6]:
import xmlschema

In [8]:
xml_validator = xmlschema.XMLSchema11(str((mdd_path/"audit_reg.xsd").absolute()))


In [9]:
xml_validator.validate()

XMLSchema11(name='audit_reg.xsd', namespace='')

In [12]:
engine = Engine()
parser = etree.XMLParser(remove_blank_text=True)
f = str((engine.mdd_path / 'exemples_metier' / 'cas_test_audit_maison_1_v0.xml'))
audit = etree.parse(f, parser)

In [14]:
xml_validator.validate(audit)

XMLSchemaChildrenValidationError: failed validating <Element donnee_entree at 0x1c647900d80> with Xsd11Group(model='all', occurs=[1, 1]):

Reason: Unexpected child with tag 'enum_type_pose_id' at position 13. Tag 'enum_etat_composant_id' expected.

Schema:

  <xs:complexType xmlns:xs="http://www.w3.org/2001/XMLSchema">
      <xs:all>
          <xs:element name="description" type="xs:string" minOccurs="0" nillable="true">
              <xs:annotation>
                  <xs:appinfo source="logement/enveloppe/porte_collection/porte/donnee_entree/description" />
                  <xs:documentation>description textuelle de l'objet</xs:documentation>
              </xs:annotation>
          </xs:element>
          <xs:element name="reference">
              <xs:annotation>
                  <xs:documentation>reference projet de l'objet (cette référence permet de faire d'éventuels liens entre objets). La codification et utilisation des références peut différer entre logiciels</xs:documentatio

In [7]:
import controle_coherence
from controle_coherence.utils import reload_package,convert_xml_text
reload_package(controle_coherence)
import controle_coherence

from controle_coherence.controle_coherence import EngineDPE,Report
from controle_coherence.utils import reload_package,convert_xml_text,element_to_value_dict
from controle_coherence.assets_dpe import table_interp,tv_table_to_value,tv_var_to_modele,\
expected_pt_liaison,default_isol_for_pt_isol_mais_inconnu,modele_verification_paths,bool_trad_fr,systeme_to_cle_repartition
from controle_coherence.utils import set_xml_values_from_dict
engine = EngineDPE()
report =Report()
self = engine


In [16]:
logement = dpe.find('logement')

In [17]:
logement

<Element logement at 0x227e16d1b40>

In [13]:
import uuid
VALID_CASES = ['cas_test_immeuble_1_valid.xml',
               'cas_test_immeuble_1_valid_avec_reference.xml',  'cas_test_immeuble_1_valid_avec_reference_v2.1.xml', 'cas_test_immeuble_1_valid_ecs_solaire_expert.xml',
               "cas_test_immeuble_1_valid_v1.xml", 'cas_test_tertiaire_1_vierge_valid.xml',
               'cas_test_immeuble_1_valid_reseau_chaleur.xml',
               'cas_test_immeuble_1_valid_generation_cascade.xml',
               'cas_test_immeuble_1_valid_ss_masque_solaire.xml', 'cas_test_tertiaire_1_neuf_valid.xml',
               'cas_test_appartement_a_partir_immeuble_valid.xml']
for el in VALID_CASES:
    parser = etree.XMLParser(remove_blank_text=True)
    f = str((engine.mdd_path / 'exemples_metier' / el))
    dpe = etree.parse(f, parser)
    
    for de in dpe.iterfind('*//donnee_entree'):
        reference=de.find('reference')
        if reference is None:
            reference=SubElement(de,'reference')
        reference.text = str(uuid.uuid4())
    dqsdq

NameError: name 'dqsdq' is not defined

In [22]:
for el in report['warning_saisie']:
    print(el['message'])


le type d'isolation des planchers bas saisi dans les ponts thermiques est : Non isolé
ceci est incohérent avec le type d'isolation saisi pour les planchers bas du logement : ITE

objets concernés : 
pont thermique Plancher bas/mur
pont thermique Plancher haut/mur

le type d'isolation des planchers hauts saisi dans les ponts thermiques est : Non isolé
ceci est incohérent avec le type d'isolation saisi pour les planchers hauts du logement : ITE

objets concernés : 
pont thermique Plancher bas/mur
pont thermique Plancher haut/mur

pour le calcul du b d'une paroi de type mur celle ci est considérée isolé : Oui
alors que l'isolation de la paroi de type mur est {2: 'Non isolé'}
Cet avertissement peut être ignoré pour le cas particulier où l'isolation des parois donnant sur le local non chauffé est inhomogène et le type d'isolation de cette paroi est minoritaire.

objets concernés : 
mur adjacent lnc U saisi

une baie vitrée orientée {2: 'Nord'}
ne correspond à aucun mur donnant sur l'exteri

In [7]:
    engine = Engine()

    parser = etree.XMLParser(remove_blank_text=True)
    f = str((engine.mdd_path / 'exemples_metier' / 'cas_test_appt_1.xml'))
    dpe = etree.parse(f, parser)

    report = Report()

    self = engine

    self.controle_coherence_tv_values_simple(dpe, report)

In [13]:
    # test doublage

    f = str((engine.mdd_path / 'exemples_metier' / 'cas_test_immeuble_1_valid.xml'))
    dpe = etree.parse(f, parser)
    mur_avec_doublage = list(dpe.iterfind('*//mur'))[1]
    mur_avec_doublage.find('*//umur').text = '0.77'
    report = Report()
    self.controle_coherence_tv_values_simple(dpe, report)
    assert (len(report.report['erreur_logiciel']) == 1)
    mur_avec_doublage.find('*//umur').text = '0.94'
    report = Report()
    self.controle_coherence_tv_values_simple(dpe, report)
    assert (len(report.report['erreur_logiciel']) == 0)
    # cas avec une période d'isolation récente on vérifie que la saisie par tv défaut passe
    mur_avec_doublage.find('*//umur').text = '0.4'
    mur_avec_doublage.find('*//tv_umur_id').text = '42'
    report = Report()
    self.controle_coherence_tv_values_simple(dpe, report)
    assert (len(report.report['erreur_logiciel']) == 0)
    # cas avec une période d'isolation récente on vérifie que la saisie par tv défaut passe avec application du doublage
    mur_avec_doublage.find('*//umur').text = '0.36'
    mur_avec_doublage.find('*//tv_umur_id').text = '42'
    report = Report()
    self.controle_coherence_tv_values_simple(dpe, report)
    assert (len(report.report['erreur_logiciel']) == 0)
    # cas avec une période d'isolation récente on vérifie que la saisie est rejetée si U=U0
    mur_avec_doublage.find('*//umur').text = '0.94'
    mur_avec_doublage.find('*//tv_umur_id').text = '42'
    report = Report()
    self.controle_coherence_tv_values_simple(dpe, report)
    assert (len(report.report['erreur_logiciel']) == 1)

In [11]:
mur_avec_doublage.find('*//tv_umur_id').text 

'42'

In [15]:
np.isclose(0.36,0.36900369003690037,atol=0.01)

True

In [8]:
print(report['erreur_logiciel'][0]['message'])

IndexError: list index out of range

In [8]:
1/((1/2.75)+0.7)

0.9401709401709403

In [8]:
1/((1/0.94)+0.21)

0.7850342408551862

In [9]:
len(report['erreur_logiciel'])

3

In [12]:
for el in report.report['erreur_logiciel']:
    print(el['message'])


la valeur de b ne correspond pas à la valeur attendue de 
la table de valeur :

valeur : 0.35
valeur table : 0.4 
tv_coef_reduction_deperdition_id : 94


la valeur de b ne correspond pas à la valeur attendue de 
la table de valeur :

valeur : 0.45
valeur table : 0.8 
tv_coef_reduction_deperdition_id : 73


                                                la valeur de umur ne correspond pas à la valeur attendue de 
                                                la table de valeur OU à la valeur attendue pour une paroi avec doublage.
                                                
                                                valeur : 2
                                                valeur table : 2.5 
                                                valeur attendue avec doublage  : 1.4084507042253522 
                                                tv_umur_id : 1
                                                valeur umur0 : 2 
                                                

     

In [5]:

report = Report()
engine.controle_coherence_pont_thermique(dpe, report)

dpe.find('*//enum_methode_application_dpe_log_id').text='13'

In [11]:
list(dpe.iterfind('*//enum_cfg_isolation_lnc_id'))

[<Element enum_cfg_isolation_lnc_id at 0x1b1d95bac80>,
 <Element enum_cfg_isolation_lnc_id at 0x1b1d9941b80>,
 <Element enum_cfg_isolation_lnc_id at 0x1b1d9941a80>,
 <Element enum_cfg_isolation_lnc_id at 0x1b1d9941300>,
 <Element enum_cfg_isolation_lnc_id at 0x1b1d99419c0>]

In [17]:
    engine = Engine()

    parser = etree.XMLParser(remove_blank_text=True)
    f = str((engine.mdd_path / 'exemples_metier' / 'cas_test_appartement_a_partir_immeuble_valid.xml'))
    dpe = etree.parse(f, parser)
    engine.controle_coherence_surfaces(dpe, report)
    assert (len(report.error_software) == 0)
    # si on passe en appt normal toutes les surfaces sont des surfaces d'immeuble donc erreur
    dpe.find('*//enum_methode_application_dpe_log_id').text='2'
    engine.controle_coherence_surfaces(dpe,report)
    assert(len(report.error_software)>=0)

AssertionError: 

In [18]:
for el in report.error_software:
    print(el['message'])


        Pour un DPE à l'appartement issu d'un DPE immeuble il doit être précisé cle_repartition_ch pour un objet de type installation_chauffage 
        Ceci afin de spécifier la part de consommation immeuble à attribuer au logement. 
        

        Pour un DPE à l'appartement issu d'un DPE immeuble il doit être précisé cle_repartition_ecs pour un objet de type installation_ecs 
        Ceci afin de spécifier la part de consommation immeuble à attribuer au logement. 
        

        Pour un DPE à l'appartement issu d'un DPE immeuble il doit être précisé cle_repartition_ventilation pour un objet de type ventilation 
        Ceci afin de spécifier la part de consommation immeuble à attribuer au logement. 
        

        Pour un DPE à l'appartement issu d'un DPE immeuble il doit être précisé cle_repartition_clim pour un objet de type climatisation 
        Ceci afin de spécifier la part de consommation immeuble à attribuer au logement. 
        

        Pour un DPE à l'apparteme

In [14]:
methode_dpe = int(dpe.find('*//enum_methode_application_dpe_log_id').text)
methode_application_dpe = self.enum_table['methode_application_dpe_log'].loc[methode_dpe].methode_application_dpe

is_dpe_appartement_immeuble = methode_application_dpe =='dpe appartement généré à partir des données DPE immeuble'

if is_dpe_appartement_immeuble: 
    for container_name,cle_repartition in systeme_to_cle_repartition.items():
        for container_object in dpe.iterfind(f'*//{container_name}'):
            el_cle_repartition = container_object.find(f'*//{cle_repartition}')
            if el_cle_repartition is None:
                msg = f"""
        Pour un DPE à l'appartement issu d'un DPE immeuble il doit être précisé {cle_repartition} pour un objet de type {container_name} 
        Ceci afin de spécifier la part de consommation immeuble à attribuer au logement. 
        """
                related_objects = [container_object]
                report.generate_msg(msg, msg_type='erreur_logiciel',
                                    msg_theme='error_cle_repartition',
                                    related_objects=related_objects,
                                    msg_importance='blocker_as_warning')

In [16]:
for el in report.report['erreur_logiciel']:
    print(el['message'])


        Pour un DPE à l'appartement issu d'un DPE immeuble il doit être précisé cle_repartition_ch pour un objet de type installation_chauffage 
        Ceci afin de spécifier la part de consommation immeuble à attribuer au logement. 
        

        Pour un DPE à l'appartement issu d'un DPE immeuble il doit être précisé cle_repartition_ecs pour un objet de type installation_ecs 
        Ceci afin de spécifier la part de consommation immeuble à attribuer au logement. 
        

        Pour un DPE à l'appartement issu d'un DPE immeuble il doit être précisé cle_repartition_ventilation pour un objet de type ventilation 
        Ceci afin de spécifier la part de consommation immeuble à attribuer au logement. 
        

        Pour un DPE à l'appartement issu d'un DPE immeuble il doit être précisé cle_repartition_clim pour un objet de type climatisation 
        Ceci afin de spécifier la part de consommation immeuble à attribuer au logement. 
        

        Pour un DPE à l'apparteme

KeyError: 'error_cle_repartition bad message theme'

In [11]:
methode_application_dpe

'dpe immeuble collectif'

In [12]:
self.enum_table['methode_application_dpe_log'].methode_application_dpe.value_counts()

dpe immeuble collectif                                                                           9
dpe appartement individuel                                                                       6
dpe appartement généré à partir des données DPE immeuble                                         6
dpe issu d'une étude energie environement réglementaire RE2020 bâtiment : appartement            4
dpe issu d'une étude thermique réglementaire RT2012 bâtiment : appartement                       4
dpe maison individuelle                                                                          1
dpe issu d'une étude thermique réglementaire RT2012 bâtiment : maison individuelle               1
dpe issu d'une étude energie environement réglementaire RE2020 bâtiment : immeuble               1
dpe issu d'une étude energie environement réglementaire RE2020 bâtiment : maison individuelle    1
dpe issu d'une étude thermique réglementaire RT2012 bâtiment : immeuble                          1
Name: meth

In [7]:
dpe.find('*//enum_methode_application_dpe_logement_id').text

for installation in dpe.iterfind('*//installation_chauffage'):
    pass

AttributeError: 'NoneType' object has no attribute 'text'

In [23]:
for el in report.report['warning_saisie']:
    print(el['message'])


le type d'isolation des planchers bas saisi dans les ponts thermiques est : Non isolé
ceci est incohérent avec le type d'isolation saisi pour les planchers bas du logement : ITE


le type d'isolation des planchers hauts saisi dans les ponts thermiques est : Non isolé
ceci est incohérent avec le type d'isolation saisi pour les planchers hauts du logement : ITE



In [5]:
self = engine

In [6]:
engine = Engine()

parser = etree.XMLParser(remove_blank_text=True)
f = str((engine.mdd_path / 'exemples_metier' / 'cas_test_immeuble_1_valid_avec_reference.xml'))
dpe = etree.parse(f, parser)

In [21]:
len("""
1 Chemin du Château de Paris,
Saint-Remy-en-Bouzemont-Saint-Genest-et-Isson - 51513
""")

85

In [22]:
len('tribu_3cl_8.8.8.0')

17

In [9]:
report = Report()

In [12]:
engine.controle_coherence_tv_values_simple(dpe,report)

In [14]:
report.report

{'validation_xsd': {},
 'warning_logiciel': [],
 'warning_saisie': [],
 'erreur_logiciel': [],
 'erreur_saisie': [],
 'notification': [],
 'message_principal': ''}

In [7]:
'1.1': {
    'xsd_file': 'DPEV1(OBSOLETE).xsd',
    'start_date': '2021-10-04',
    'end_date': '2022-01-04',
    'end_date_compare_now': '2022-01-04',
    'end_date_edit': '2022-03-01', # date limite pour les rééditions
    'is_future': False,
    'version_controle_coherence': 0,

<Element dpe_a_remplacer at 0x25e3f4c1a80>

In [ ]:
    def validation_version_dpe_et_dates(self, dpe, report):
        date_etablissement_dpe = dpe.find('*//date_etablissement_dpe').text
        date_etablissement_dpe = datetime.fromisoformat(date_etablissement_dpe)

        # on compare avec la date d'établissement dpe pour la date de fin de validité pour permettre le dépôt de DPE en retard
        # on compare avec la date de transmission pour la date de début de validité.
        # c'est la date de transmission qui fait foi.

        version_id_str = dpe.find('*//enum_version_id').text
        start_date_version = datetime.fromisoformat(versions_dpe_cfg[version_id_str]['start_date'])
        end_date_version = datetime.fromisoformat(versions_dpe_cfg[version_id_str]['end_date'])
        end_date_compare_now_version = datetime.fromisoformat(versions_dpe_cfg[version_id_str]['end_date_compare_now'])
        end_date_edit = versions_dpe_cfg[version_id_str].get('end_date_edit')
        now = datetime.now()

        if end_date_edit is not None:
            end_date_edit = datetime.fromisoformat(end_date_edit)
            dpe_a_remplacer = dpe.find('administratif/dpe_a_remplacer')
            if dpe_a_remplacer is not None:
                is_valid_reedit = (date_etablissement_dpe <= end_date_edit) & (now <= end_date_edit)
            else:
                is_valid_reedit = False
        else:
            is_valid_reedit = False

        # invalidation de tous les DPE qui déclarent une date d'etablissement de DPE future
        if now < date_etablissement_dpe:
            msg = f"""le DPE est fourni avec une date d'établissement du DPE supérieure à la date d'aujourd'hui.
            date d'établissement du DPE déclarée : {date_etablissement_dpe}
            date du jour : {now.date().strftime('%Y-%m-%d')}

            """
            report.generate_msg(msg, msg_type='erreur_saisie',
                                msg_theme='invalid_date_etablissement',
                                related_objects=[dpe.find('*//enum_version_id'), dpe.find('*//date_etablissement_dpe')],
                                msg_importance='blocker')

        is_old_version = date_etablissement_dpe > end_date_version
        is_old_version_now = now > end_date_compare_now_version
        is_valid_version = (date_etablissement_dpe <= end_date_version) & (now >= start_date_version) & (now <= end_date_compare_now_version)
        is_valid_version = is_valid_version | is_valid_reedit  # soit la réédition est valide soit le dpe est valide pour première transmission
        is_future_version = now < start_date_version

        if is_old_version & (not is_valid_version):
            msg = f"""le DPE est fourni avec un numéro de version obsolète : {version_id_str}.
        date d'expiration de la version : {end_date_version}
        Votre logiciel DPE doit être mis à jour. 
        les versions suivantes sont acceptées pour dépôt. {current_valid_versions}
        """
            report.generate_msg(msg, msg_type='erreur_logiciel',
                                msg_theme='out_of_date_version',
                                related_objects=[dpe.find('*//enum_version_id'), dpe.find('*//date_etablissement_dpe')],
                                msg_importance='blocker')
        if is_old_version_now & (not is_valid_version):
            msg = f"""le DPE est fourni avec un numéro de version obsolète : {version_id_str}.
        date d'expiration de la version : {end_date_compare_now_version}
        Votre logiciel DPE doit être mis à jour. 
        les versions suivantes sont acceptées pour dépôt. {current_valid_versions}
        """
            report.generate_msg(msg, msg_type='erreur_logiciel',
                                msg_theme='out_of_date_version',
                                related_objects=[dpe.find('*//enum_version_id'), dpe.find('*//date_etablissement_dpe')],
                                msg_importance='blocker')

        elif is_future_version & (not is_valid_version):
            msg = f"""le DPE est fourni avec un numéro de version en cours de développement non autorisée pour dépôt. : {version_id_str}
        Veuillez contacter votre Editeur de Logiciel. 
        les versions suivantes sont acceptées pour dépôt. {current_valid_versions}
        """
            report.generate_msg(msg, msg_type='erreur_logiciel',
                                msg_theme='out_of_date_version',
                                related_objects=[dpe.find('*//enum_version_id'), dpe.find('*//date_etablissement_dpe')],
                                msg_importance='blocker')

        is_old_version = is_old_version & (not is_valid_version)
        is_old_version = is_old_version | (is_old_version_now & (not is_valid_version))

        return is_old_version

In [ ]:
assert(report['erreur_logiciel']==2)


In [13]:
len(report['erreur_logiciel'])

2

In [18]:
for prop in ['conso_ecs', 'conso_ecs_depensier']:
    prop_0_list = dpe.xpath(f'*//installation_ecs/donnee_intermediaire/{prop}[number(text())=0]')
    for prop_0 in prop_0_list:
        fecs = prop_0.getparent().find('fecs')
        installation = prop_0.getparent().getparent()
        if fecs is not None:
            fecs = float(fecs.text)
        else:
            fecs = 0

In [19]:
prop_0

<Element conso_ecs_depensier at 0x2c5ccfd2680>

In [20]:
fecs

0.8

In [10]:
prop_0_list

[<Element conso_ecs_depensier at 0x2c5ccfd2680>]

In [6]:
assert(len(report['erreur_logiciel'])==2)

AssertionError: 

In [18]:
report['erreur_logiciel']

[]

In [6]:

report=engine.run_controle_coherence(dpe)

In [12]:
dpe.find('*//fecs_saisi').text='0.42'
dpe.find('*//fch_saisi').text='0.42'

In [13]:
report=engine.run_controle_coherence(dpe)
assert(len(report['erreur_logiciel'])==2)

In [14]:
for el in report['erreur_logiciel']:
    print(el['message'])


la valeur de fecs_saisi ne correspond pas à fecs
fecs_saisi : 0.42 
fecs : 0.8


la valeur de fch_saisi ne correspond pas à fch
fch_saisi : 0.42 
fch : 0.9



In [8]:
report

{'validation_xsd': {'valid': True, 'error_log': ['']},
 'warning_logiciel': [],
 'warning_saisie': [{'message': "\npour le calcul du b d'une paroi de type mur celle ci est considérée isolé : Oui\nalors que l'isolation de la paroi de type mur est {2: 'Non isolé'}\nCet avertissement peut être ignoré pour le cas particulier où l'isolation des parois donnant sur le local non chauffé est inhomogène et le type d'isolation de cette paroi est minoritaire.\n\nobjets concernés : \nMur exterieur sud",
   'thematique': "saisie incohérente d'un élément d'enveloppe.",
   'objets_concerne': ['/dpe/logement/enveloppe/mur_collection/mur[1]/donnee_entree/enum_type_isolation_id'],
   'importance': 'majeur'},
  {'message': "\npour le calcul du b d'une paroi de type mur celle ci est considérée isolé : Oui\nalors que l'isolation de la paroi de type mur est {2: 'Non isolé'}\nCet avertissement peut être ignoré pour le cas particulier où l'isolation des parois donnant sur le local non chauffé est inhomogène et

In [7]:
engine = Engine()

parser = etree.XMLParser(remove_blank_text=True)
f = str((engine.mdd_path / 'exemples_metier' / 'cas_test_immeuble_1_valid_generation_cascade.xml'))
dpe = etree.parse(f, parser)
report = Report()

for prop in ['rendement_generation', 'conso_ch_depensier', 'conso_ch']:
    for el in dpe.iterfind(f'*//generateur_chauffage//{prop}'):
        el.text = str(0)

engine.controle_coherence_generateur_cascade(dpe, report)
assert(len(report.error_software) == 3)

dpe.find('*//priorite_generateur_cascade').text = '2'
report = Report()
engine.controle_coherence_generateur_cascade(dpe, report)
assert(len(report.error_software) == 1)

AttributeError: 'Engine' object has no attribute 'controle_coherence_generateur_cascade'

In [8]:
self = engine
cascades = [el for el in dpe.iterfind('*//priorite_generateur_cascade')]
if len(cascades) > 0:
    cascades_num = [int(el.text) for el in cascades]
    max_cascade = max(cascades_num)
    if max_cascade > 1 and 1 not in cascades_num:
        msg = """
un générateur à combustion en cascade est déclaré avec priorite_generateur_cascade en priorité 2 ou +(non prioritaire) alors qu'aucun générateur en cascade n'est
déclaré prioritaire (1)
"""
        report.generate_msg(msg, msg_type='erreur_logiciel',
                            msg_theme='error_generateur_cascade_order',
                            related_objects=cascades,
                            msg_importance='blocker_as_warning')

for prop in ['rendement_generation', 'conso_ch_depensier', 'conso_ch']:

    prop_0_list = dpe.xpath(f'*//generateur_chauffage//donnee_intermediaire//{prop}[number(text())=0]')

    for prop_0 in prop_0_list:
        generateur = prop_0.getparent().getparent()
        installation = generateur.getparent().getparent()

'1'

In [29]:
report.error_software

[{'message': "\n        un générateur à combustion en cascade est déclaré avec priorite_generateur_cascade en priorité 2 ou +(non prioritaire) alors qu'aucun générateur en cascade n'est\n        déclaré prioritaire (1)\n        ",
  'thematique': 'error_generateur_cascade_order',
  'objets_concerne': [<Element priorite_generateur_cascade at 0x279df30f4c0>,
   <Element priorite_generateur_cascade at 0x279df30f600>],
  'importance': 'blocker_as_warning'},
 {'message': "\n        le champ rendement_generation vaut 0 pour un générateur de chauffage de type {85: 'Chaudière gaz classique avant 1981'}\n        Ceci n'est autorisé que dans le cas d'un générateur à combustion en cascade avec priorité qui est non prioritaire.\n        Le générateur en question ne correspond pas à cette configuration.\n        ce générateur est prioritaire. ordre de priorité : 1.",
  'thematique': 'error_generateur_cascade_valeur_0',
  'objets_concerne': [<Element generateur_chauffage at 0x279df30fc00>,
   <Eleme

In [11]:
len(report['erreur_logiciel'])

34

In [12]:
len(report['erreur_saisie'])

0

In [9]:
engine = Engine()
report =Report()
self = engine
parser = etree.XMLParser(remove_blank_text=True)
f = str((engine.mdd_path/'exemples_metier'/'cas_test_immeuble_1_valid.xml'))
dpe = etree.parse(f, parser)
dpe.find('*//ep_conso/ep_conso_5_usages_m2').text='421'
dpe.find('*//ep_conso/classe_bilan_dpe').text='G'
dpe.find('*//classe_emission_ges').text="A"
dpe.find('*//emission_ges_5_usages_m2').text="5"

engine.controle_coherence_etiquette(dpe,report)

assert(len(report.error_software)==0)

dpe.find('*//enum_zone_climatique_id').text="7"
engine.controle_coherence_etiquette(dpe,report)

assert(len(report.error_software)==0)
dpe.find('*//enum_classe_altitude_id').text="3"
engine.controle_coherence_etiquette(dpe,report)

assert(len(report.error_software)==1)

report =Report()

dpe.find('*//ep_conso/ep_conso_5_usages_m2').text='190'
dpe.find('*//ep_conso/classe_bilan_dpe').text='D'
engine.controle_coherence_etiquette(dpe,report)

assert(len(report.error_software)==0)

dpe.find('*//classe_emission_ges').text="E"
dpe.find('*//emission_ges_5_usages_m2').text="50"

engine.controle_coherence_etiquette(dpe,report)

assert(len(report.error_software)==1)

report =Report()
dpe.find('*//classe_emission_ges').text="D"
engine.controle_coherence_etiquette(dpe,report)
assert(len(report.error_software)==2)


In [11]:
print(report.error_software[1]['message'])

la classe energétique bilan du DPE ne correspond pas à la valeur d'emission 5 usages calculée 
et la valeur de consommation energie primaire 5 usages
classe energétique bilan du DPE fournie : D
emission GES 5 usage fournie : 50.0
consommation EP 5 usage fournie : 190.0
classe energétique bilan du DPE attendue : E 
car soit 
la consommation EP est dans l'intervalle : 250,390
et l'emission GES est inférieure à 80
ou 
l'emission GES est dans l'intervalle : 50,80
et la consommation EP est inférieur à 390



In [8]:
from controle_coherence.assets import *

In [109]:

classe_bilan_dpe=dpe.find('*//classe_bilan_dpe').text
classe_emission_ges=dpe.find('*//classe_emission_ges').text

ep_conso_5_usages_m2=float(dpe.find('*//ep_conso/ep_conso_5_usages_m2').text)
emission_ges_5_usages_m2=float(dpe.find('*//emission_ges/emission_ges_5_usages_m2').text)
enum_zone_climatique_id = int(dpe.find('*//enum_zone_climatique_id').text)
enum_classe_altitude_id = int(dpe.find('*//enum_classe_altitude_id').text)

# dans le cas d'un bâtiment en altitude pour les zones climatiques H1b, H1c et H2d les seuils E,F et G sont changés
if (enum_zone_climatique_id in zones_climatiques_altitude) and (enum_classe_altitude_id==3):
    seuils_energie=seuils_energie_altitude
    seuils_ges=seuils_ges_altitude


for etiquette,(min_value_ges,max_value_ges) in seuils_ges.items():
    if emission_ges_5_usages_m2<max_value_ges:
        classe_emission_ges_expected=etiquette
        break

# test si la classe emission ges est bonne
if classe_emission_ges_expected != classe_emission_ges:
    msg = f"""la classe GES du DPE ne correspond pas à la valeur d'emission GES 5 usages calculée 
    classe étiquette GES fournie : {classe_emission_ges}
    emission GES 5 usage fournie : {emission_ges_5_usages_m2}
    classe étiquette GES attendue : {classe_emission_ges_expected} (intervalle {min_value_ges},{max_value_ges})
    """
    report.generate_msg(msg, msg_type='erreur_logiciel',
                    msg_theme='bad_etiquette_calculation',
                    related_objects=[dpe.find('*//classe_emission_ges'),
                                     dpe.find('*//emission_ges/emission_ges_5_usages_m2')],
                    msg_importance='blocker')
    
for etiquette,(min_value_ener,max_value_ener) in seuils_energie.items():
    if ep_conso_5_usages_m2<max_value_ener:
        classe_conso_energie_expected=etiquette
        break
        
classe_bilan_dpe_expected = max(classe_emission_ges_expected,classe_conso_energie_expected)

# test si la classe bilan energétique est bonne
if classe_bilan_dpe_expected != classe_bilan_dpe:
    msg = f"""la classe energétique bilan du DPE ne correspond pas à la valeur d'emission 5 usages calculée 
    et la valeur de consommation energie primaire 5 usages
    classe energétique bilan du DPE fournie : {classe_bilan_dpe}
    emission GES 5 usage fournie : {emission_ges_5_usages_m2}
    consommation EP 5 usage fournie : {ep_conso_5_usages_m2}
    classe energétique bilan du DPE attendue : {classe_conso_energie_expected} 
    car soit 
    la consommation EP est dans l'intervalle : {seuils_energie[classe_bilan_dpe_expected][0]},{seuils_energie[classe_bilan_dpe_expected][1]}
    et l'emission GES est inférieure à {seuils_ges[classe_bilan_dpe_expected][1]}
    ou 
    l'emission GES est dans l'intervalle : {seuils_ges[classe_bilan_dpe_expected][0]},{seuils_ges[classe_bilan_dpe_expected][1]}
    et la consommation EP est inférieur à {seuils_energie[classe_bilan_dpe_expected][1]}
    """
    report.generate_msg(msg, msg_type='erreur_logiciel',
                    msg_theme='bad_etiquette_calculation',
                    related_objects=[dpe.find('*//classe_emission_ges'),
                                     dpe.find('*//emission_ges/emission_ges_5_usages_m2')],
                    msg_importance='blocker')

In [101]:
print(report.error_software[0]['message'])

la classe GES du DPE ne correspond pas à la valeur d'emission GES 5 usages calculée 
    classe étiquette GES fournie : D
    emission GES 5 usage fournie : 22.88
    classe étiquette GES attendue : C (intervalle 11,30)
    


In [102]:
print(report.error_software[1]['message'])

la classe energétique bilan du DPE ne correspond pas à la valeur d'emission 5 usages calculée 
    et la valeur de consommation energie primaire 5 usages
    classe energétique bilan du DPE fournie : F
    emission GES 5 usage fournie : 22.88
    consommation EP 5 usage fournie : 421.0
    classe energétique bilan du DPE attendue : G 
    car soit 
    la consommation EP est dans l'intervalle : 420,inf
    et l'emission GES est inférieure à inf
    ou 
    l'emission GES est dans l'intervalle : 100,inf
    et la consommation EP est inférieur à inf
    


In [103]:
for etiquette,(min_value,max_value) in seuils_energie.items():
    if ep_conso_5_usages_m2<max_value:
        classe_conso_energie_expected=etiquette
        break

In [57]:
classe_conso_energie_expected

'G'

In [30]:
enum_zone_climatique_id in 

'1'

In [21]:
import numpy as np

In [28]:
seuils_energie = {'A':[-np.inf,70],
                 'B':[70,110],
                  'C':[110,180],
                  'D':[180,250],
                 }
seuils_energie_altitude = seuils_energie.copy()

seuils_energie.update({'E':[250,330],
                      'F':[330,420],
                      'G':[420,np.inf]
}
)
seuils_energie_altitude.update({'E':[250,390],
                      'F':[390,500],
                      'G':[500,np.inf]
}
)


seuils_ges = {'A':[-np.inf,6],
                 'B':[6,11],
                  'C':[11,30],
                  'D':[30,50],
                 }
seuils_ges_altitude = seuils_ges.copy()

seuils_ges.update({'E':[50,70],
                      'F':[70,100],
                      'G':[100,np.inf]
}
)
seuils_ges_altitude.update({'E':[50,80],
                      'F':[80,110],
                      'G':[110,np.inf]
}
)


In [5]:
dpe.find('*//b').text = '0'

In [6]:
tv = dpe.find('*//tv_coef_reduction_deperdition_id')
tv.getparent().remove(tv)

In [7]:
engine.controle_coherence_enveloppe(dpe,report)

In [8]:
print(report.warning_input[0]['message'])


la valeur de b pour une paroi de type mur vaut 0
cette valeur n'est acceptable que dans certaines configurations :
la surface_aue de la paroi vaut 0 
surface_aue n'est pas déclaré pour la paroi correspondante.
ou la valeur de tv_coef_reduction_deperdition_id correspond à une valeur de la table =0.
tv_coef_reduction_deperdition_id n'est pas déclaré pour la paroi correspondante. 



In [14]:
engine.valeur_table_dict['tv_coef_reduction_deperdition_id'][int(tv_coef_reduction_deperdition_id.text)]['b']==1

True

In [7]:
type_generateur_id = [int(el.text) for el in dpe.iterfind('*//enum_type_generateur_ch_id')]

In [11]:
type_generateur_id.append(143)

True

In [11]:
dpe = etree.parse(f, parser)
adresse_proprietaire_commun = dpe.find('*//adresse_proprietaire_installation_commune')
adresse_proprietaire_commun.getparent().remove(adresse_proprietaire_commun)
dpe.find('administratif//ban_type').text = 'municipality'
engine.run_controle_coherence(dpe)


{'validation_xsd': {'valid': True, 'error_log': ['']},
 'warning_logiciel': [],
 'warning_saisie': [{'message': "\nLe resultat du géocodage n'est pas précis au numéro de rue mais à une échelle plus imprécise municipality\nCeci est anormal dans la plupart des cas. Essayez de préciser l'adresse au niveau du numéro de rue si celui-ci est bien référencé dans la Base d'Adresse Nationale\nvous pouvez vous aider de cette carte :  https://adresse.data.gouv.fr/base-adresse-nationale  \n",
   'thematique': "l'adresse du bien semble imprécise ou érronée.",
   'objets_concerne': ['/dpe/administratif/geolocalisation/adresses/adresse_proprietaire',
    '/dpe/administratif/geolocalisation/adresses/adresse_proprietaire/ban_type'],
   'importance': 'majeur'}],
 'erreur_logiciel': [],
 'erreur_saisie': [],
 'notification': [],
 'message_principal': 'CONTROLE COHERENCE DU MODELE NON EFFECTUE SUR DPE LOGEMENT EXISTANT ALLEGE.\npour tester le controle de cohérence de la version complète,  enum_version_id d

In [9]:
dpe.find('administratif//ban_type').text=''

In [7]:
report.report

{'validation_xsd': {},
 'warning_logiciel': [],
 'warning_saisie': [],
 'erreur_logiciel': [],
 'erreur_saisie': [],
 'notification': [],
 'message_principal': ''}

In [7]:
report.warning_input

[{'message': "\nLe resultat du géocodage n'est pas précis au numéro de rue mais à une échelle plus imprécise housenumber\nCeci est anormal dans la plupart des cas. Essayez de préciser l'adresse au niveau du numéro de rue si celui-ci est bien référencé dans la Base d'Adresse Nationale\nvous pouvez vous aider de cette carte :  https://adresse.data.gouv.fr/base-adresse-nationale  \n",
  'thematique': 'address_anomaly',
  'objets_concerne': [<Element adresse_proprietaire at 0x202161908c0>,
   <Element ban_type at 0x20216243440>],
  'importance': 'major'},
 {'message': "\nLe resultat du géocodage n'est pas précis au numéro de rue mais à une échelle plus imprécise housenumber\nCeci est anormal dans la plupart des cas. Essayez de préciser l'adresse au niveau du numéro de rue si celui-ci est bien référencé dans la Base d'Adresse Nationale\nvous pouvez vous aider de cette carte :  https://adresse.data.gouv.fr/base-adresse-nationale  \n",
  'thematique': 'address_anomaly',
  'objets_concerne': [

In [12]:
print(report['warning_saisie'][0]['message'])


L'adresse adresse_proprietaire n'as pas été géocodée. Vérifier que l'adresse saisie 3 rue roland martin 94500 Champigny Sur Marne est bien valide.
Il est possible que l'adresse adresse_proprietaire ne soit pas référencé dans la Base d'Adresse Nationale.
vous pouvez le vérfier grâce à cette carte interactive https://adresse.data.gouv.fr/base-adresse-nationale       
si votre adresse existe bien dans la Base d'Adresse Nationale, tentez de relancer le géocodage depuis le logiciel DPE.
        l'adresse suivante est le meilleur résultat trouvé par l'API adresse (https://geo.api.gouv.fr/adresse)       
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [2.485764, 48.820722]}, 'properties': {'label': '3 Rue Roland Martin 94500 Champigny-sur-Marne', 'score': 0.9665599999999999, 'housenumber': '3', 'id': '94017_8243_00003', 'name': '3 Rue Roland Martin', 'postcode': '94500', 'citycode': '94017', 'x': 662242.87, 'y': 6857973.84, 'city': 'Champigny-sur-Marne', 'context': '94, Val-

In [5]:
self.controle_coherence_variables_requises(dpe,report)

In [6]:
report.error_software

[{'message': '\nles champs suivants doivent être renseignés :\nban_id et ban_label et ban_housenumber et ban_street et ban_citycode et ban_postcode et ban_city et ban_type et ban_score et ban_x et ban_y\nlorsque la variable enum_statut_geocodage_ban_id vaut : \n{1: "adresse géocodée ban à l\'adresse"}\n',
  'thematique': 'missing_required_element',
  'objets_concerne': [<Element adresse_bien at 0x11127f430c0>,
   <Element enum_statut_geocodage_ban_id at 0x11128588a40>],
  'importance': 'blocker'},
 {'message': '\nles champs suivants doivent être renseignés :\nban_id et ban_label et ban_housenumber et ban_street et ban_citycode et ban_postcode et ban_city et ban_type et ban_score et ban_x et ban_y\nlorsque la variable enum_statut_geocodage_ban_id vaut : \n{1: "adresse géocodée ban à l\'adresse"}\n',
  'thematique': 'missing_required_element',
  'objets_concerne': [<Element adresse_proprietaire_installation_commune at 0x11127f43180>,
   <Element enum_statut_geocodage_ban_id at 0x11128588

In [76]:
dpe.find(f'*//xs:element[@name="adresse_proprietaire"]', namespaces=self.namespaces)

In [79]:
model = administratif.find(f'*//xs:element[@name="{name}"]', namespaces=self.namespaces)

In [80]:
model

<Element {http://www.w3.org/2001/XMLSchema}element at 0x178e9f65640>

In [5]:
dpe.find('administratif/enum_modele_dpe_id').text=str(3)
modele_dpe_id = int(dpe.find('administratif/enum_modele_dpe_id').text)

In [133]:
report = Report()

In [ ]:
ban_types = ['housenumber','street','locality','municipality']

In [20]:
','.join([el.tag for el in el_adresse.getchildren()])

'adresse_brut,code_postal_brut,nom_commune_brut,label_brut,enum_statut_geocodage_ban_id,ban_date_appel,ban_id,ban_label,ban_housenumber,ban_street,ban_citycode,ban_postcode,ban_city,ban_type,ban_score,ban_x,ban_y,compl_nom_residence,compl_ref_batiment,compl_etage_appartement,compl_ref_cage_escalier,compl_ref_logement'

In [14]:
ban_types = ['housenumber','street','locality','municipality']

In [31]:
dpe.find('administratif//enum_statut_geocodage_ban_id').text='2'

compl_etage_appartement = dpe.find('*//adresse_bien//compl_etage_appartement')
compl_etage_appartement.getparent().remove(compl_etage_appartement)

dpe.find('administratif//ban_score').text='0.5'
dpe.find('administratif//ban_type').text='0.5'


In [72]:
try:
    int('a')
    {'a':'b'}['c']
except (ValueError,KeyError):
    print('a')

a


In [11]:
import requests

In [22]:
r=requests.get(f'https://api-adresse.data.gouv.fr/search/?q=ronjou')

In [23]:
r.json()

{'type': 'FeatureCollection',
 'version': 'draft',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-3.421576, 47.936567]},
   'properties': {'label': "Poul Ronjou 29300 Guilligomarc'h",
    'score': 0.6647199999999999,
    'id': '29071_c168',
    'name': 'Poul Ronjou',
    'postcode': '29300',
    'citycode': '29071',
    'x': 221036.63,
    'y': 6779061.47,
    'city': "Guilligomarc'h",
    'context': '29, Finistère, Bretagne',
    'type': 'street',
    'importance': 0.31192}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [3.828251, 46.739343]},
   'properties': {'label': 'Champ Ronjou 58250 Saint-Seine',
    'score': 0.6588472727272726,
    'id': '58268_b047',
    'name': 'Champ Ronjou',
    'postcode': '58250',
    'citycode': '58268',
    'x': 763238.39,
    'y': 6626912.66,
    'city': 'Saint-Seine',
    'context': '58, Nièvre, Bourgogne-Franche-Comté',
    'type': 'street',
    'importance': 0.24732}},
  {'type': 'Featur

In [65]:
dpe.find('/administratif/enum_version_id')

<Element enum_version_id at 0x178e9c5f300>

In [61]:
for el_adresse in dpe.find('*//adresses').getchildren():
    el_statut_geocodage_ban = el_adresse.find('enum_statut_geocodage_ban_id')
    adresse_dict = element_to_value_dict(el_adresse)
    statut_geocodage_ban_id = int(el_statut_geocodage_ban.text)
    code_postal_brut = el_adresse.find('code_postal_brut').text
    adresse_brut_concat = adresse_dict['adresse_brut']+' '+code_postal_brut+' '+adresse_dict['nom_commune_brut']
    if adresse_dict['enum_statut_geocodage_ban_id'] != 1:
        
        msg = f"""
L'adresse {el_adresse.tag} n'as pas été géocodée. Vérifier que l'adresse saisie {adresse_brut_concat} est bien valide.
Il est possible que l'adresse {el_adresse.tag} ne soit pas référencé dans la Base d'Adresse Nationale.
vous pouvez le vérfier grâce à cette carte interactive https://adresse.data.gouv.fr/base-adresse-nationale       
si votre adresse existe bien dans la Base d'Adresse Nationale, tentez de relancer le géocodage depuis le logiciel DPE.
        """
        try:
            r=requests.get(f'https://api-adresse.data.gouv.fr/search/?q={adresse_brut_concat}')
            if r.status_code ==200:
                r_json = r.json()
                if len(r_json.get('features',[]))>0:
                    resp_geocode = f"""l'adresse suivante est le meilleur résultat trouvé par l'API adresse (https://geo.api.gouv.fr/adresse)       
    {r_json['features'][0]}
                    """
                
            else:
                resp_geocode=""
        except:
            resp_geocode=""
        msg+=resp_geocode
        related_objects = [el_adresse]
        report.generate_msg(msg, msg_type='warning_saisie',
                            msg_theme='address_anomaly',
                            related_objects=related_objects,
                            msg_importance='critical')
    el_result_type =  el_adresse.find('ban_type')
    if el_result_type is not None:    
        if el_result_type.text not in ban_types:
            msg=f"""
erreur dans les données de géocodage de l'adresse {el_adresse.tag}.
Le type de résultat fourni dans le champs ban_type ne correspond à aucun des résultats possible {ban_types}
            """
            related_objects = [el_adresse,el_result_type]
            report.generate_msg(msg, msg_type='erreur_logiciel',
                                msg_theme='geocoding_error',
                                related_objects=related_objects,
                                msg_importance='blocker')
    el_ban_score =  el_adresse.find('ban_score')
    if el_ban_score is not None: 
        if float(el_ban_score.text)<0.8:
            msg = f"""
Le score de géocodage de l'adresse {el_adresse.tag} est faible : {el_ban_score.text}
Est ce que l'adresse géocodée : {adresse_dict.get('ban_label',"AUCUNE ADRESSE GEOCODEE RENSEIGNEE")} correspond bien à l'adresse {el_adresse.tag}
            
            """
            related_objects = [el_adresse,el_ban_score]
            report.generate_msg(msg, msg_type='warning_saisie',
                                msg_theme='address_anomaly',
                                related_objects=related_objects,
                                msg_importance='major')

el_methode_dpe = dpe.find('*//enum_methode_application_dpe_log_id')
if el_methode_dpe is not None:
    methode_dpe_id = int(el_methode_dpe.text)
    type_batiment = self.enum_table['methode_application_dpe_log'].loc[methode_dpe_id].type_batiment
    if type_batiment =='appartement':
        compl_etage_appartement = dpe.find('*//adresse_bien//compl_etage_appartement')
        if compl_etage_appartement is None:
            msg ="""
Le DPE est un DPE de type DPE appartement.
l'etage de l'appartement n'est pas précisé dans la section adresse du DPE
            """
            
            related_objects = [dpe.find('*//adresse_bien')]
            report.generate_msg(msg, msg_type='warning_saisie',
                                msg_theme='address_anomaly',
                                related_objects=related_objects,
                                msg_importance='minor')

In [63]:
report.warning_input

[]

In [51]:
.compl_etage_appartement

<Element compl_etage_appartement at 0x178e952b540>

In [ ]:
dpe.find('*//')

In [86]:
type_generateur_ch_id

87

In [81]:
pos_prob_vol_ch_dict.get(54)

1.0

In [77]:
type_generateur_id

54

In [76]:
position_probable_volume_chauffe

In [74]:
pos_prob_vol_ch_dict

{20: 1.0,
 21: 1.0,
 22: 1.0,
 23: 1.0,
 24: 1.0,
 25: 1.0,
 26: 1.0,
 27: 1.0,
 28: 1.0,
 29: 1.0,
 30: 1.0,
 31: 1.0,
 32: 1.0,
 33: 1.0,
 34: 1.0,
 35: 1.0,
 36: 1.0,
 37: 1.0,
 38: 1.0,
 39: 1.0,
 40: 1.0,
 41: 1.0,
 42: 1.0,
 43: 1.0,
 44: 1.0,
 45: 1.0,
 46: 1.0,
 47: 1.0,
 53: 1.0,
 54: 1.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 98: 1.0,
 99: 1.0,
 100: 1.0,
 101: 1.0,
 102: 1.0,
 103: 1.0,
 104: 1.0,
 105: 1.0,
 107: 0.0,
 108: 0.0,
 109: 0.0,
 110: 0.0,
 111: 0.0,
 112: 0.0,
 119: 0.0,
 120: 0.0,
 121: 0.0,
 122: 0.0,
 123: 0.0,
 124: 0.0,
 125: 0.0,
 126: 0.0,
 140: 1.0,
 141: 1.0,
 142: 0.0}

In [73]:
position_probable_volume_chauffe

In [69]:
position_volume_chauffe

NameError: name 'position_volume_chauffe' is not defined

In [62]:
el_position_volume_chauffe

<Element position_volume_chauffe at 0x281528823c0>

'87'

{20: 1.0,
 21: 1.0,
 22: 1.0,
 23: 1.0,
 24: 1.0,
 25: 1.0,
 26: 1.0,
 27: 1.0,
 28: 1.0,
 29: 1.0,
 30: 1.0,
 31: 1.0,
 32: 1.0,
 33: 1.0,
 34: 1.0,
 35: 1.0,
 36: 1.0,
 37: 1.0,
 38: 1.0,
 39: 1.0,
 40: 1.0,
 41: 1.0,
 42: 1.0,
 43: 1.0,
 44: 1.0,
 45: 1.0,
 46: 1.0,
 47: 1.0,
 53: 1.0,
 54: 1.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 98: 1.0,
 99: 1.0,
 100: 1.0,
 101: 1.0,
 102: 1.0,
 103: 1.0,
 104: 1.0,
 105: 1.0,
 107: 0.0,
 108: 0.0,
 109: 0.0,
 110: 0.0,
 111: 0.0,
 112: 0.0,
 119: 0.0,
 120: 0.0,
 121: 0.0,
 122: 0.0,
 123: 0.0,
 124: 0.0,
 125: 0.0,
 126: 0.0,
 140: 1.0,
 141: 1.0,
 142: 0.0}

In [28]:
rpn_sup_rpint=self.enum_table['type_generateur_ch']['rpn_sup_rpint'].dropna()
for el_rpn in dpe.iterfind('*//generateur_chauffage//rpn'):
    el_rpint = el_rpn.getparent().find('rpint')
    if el_rpint is not None:
        el_type_generateur_ch=el_rpn.getparent().getparent().find('*//enum_type_generateur_ch_id')
        rpn = float(el_rpn.text)
        rpint = float(el_rpint.text)
        type_generateur_ch_id = int(el_type_generateur_ch.text)
        logic = rpn_sup_rpint.get(type_generateur_ch_id)
        if logic is not None:
            if logic == 0 and (rpn>rpint):
                related_objects = [el_rpn,el_rpint]
                msg =f"""
pour un générateur de type {self.display_enum_traduction('enum_type_generateur_ch_id',type_generateur_ch_id)}
Rpn > Rpint. Pour ce type de générateur c'est en général le contraire qui est attendu. 
Rpn :  {rpn}
Rpint :  {rpint}
"""
                report.generate_msg(msg,
                        msg_type='warning_saisie',
                        msg_theme='incoherence_systeme',
                        related_objects=related_objects,
                        msg_importance="minor"
                        )
            elif logic == 1 and (rpn<rpint):
                related_objects = [el_rpn,el_rpint]
                msg =f"""
pour un générateur de type {self.display_enum_traduction('enum_type_generateur_ch_id',type_generateur_ch_id)}
Rpn < Rpint. Pour ce type de générateur c'est en général le contraire qui est attendu. 
Rpn :  {rpn}
Rpint :  {rpint}
"""              
                report.generate_msg(msg,
                                msg_type='warning_saisie',
                                msg_theme='incoherence_systeme',
                                related_objects=related_objects,
                                msg_importance="minor"
                                )

In [44]:
 dpe.find('*//enum_classe_inertie_id').text='4'

In [46]:
report.warning_input

[{'message': "\nle système de ventilation est déclaré plus ancien que la période de construction du bâtiment\ntype ventilation : {3: 'VMC SF Auto réglable avant 1982'}\nperiode ventilation : {8: '2006-2012'}\n            ",
  'thematique': 'incoherence_systeme',
  'objets_concerne': [<Element enum_type_ventilation_id at 0x2814fff7400>],
  'importance': 'major'},
 {'message': "\npour un générateur de type {63: 'Chaudière bois plaquette 1978-1994'}\nRpn > Rpint. Pour ce type de générateur c'est en général le contraire qui est attendu. \nRpn :  0.9\nRpint :  0.81\n",
  'thematique': 'incoherence_systeme',
  'objets_concerne': [<Element rpn at 0x2815003d680>,
   <Element rpint at 0x2815003d840>],
  'importance': 'minor'},
 {'message': "\npour un générateur de type {87: 'Chaudière gaz classique 1986-1990'}\nRpn < Rpint. Pour ce type de générateur c'est en général le contraire qui est attendu. \nRpn :  0.3\nRpint :  0.81\n",
  'thematique': 'incoherence_systeme',
  'objets_concerne': [<Eleme

In [32]:
el_intermittence

<Element tv_intermittence_id at 0x2815244ea80>

In [29]:
report.warning_input

[{'message': "\nle système de ventilation est déclaré plus ancien que la période de construction du bâtiment\ntype ventilation : {3: 'VMC SF Auto réglable avant 1982'}\nperiode ventilation : {8: '2006-2012'}\n            ",
  'thematique': 'incoherence_systeme',
  'objets_concerne': [<Element enum_type_ventilation_id at 0x2814fff7400>],
  'importance': 'major'},
 {'message': "\npour un générateur de type {63: 'Chaudière bois plaquette 1978-1994'}\nRpn > Rpint. Pour ce type de générateur c'est en général le contraire qui est attendu. \nRpn :  0.9\nRpint :  0.81\n",
  'thematique': 'incoherence_systeme',
  'objets_concerne': [<Element rpn at 0x2815003d680>,
   <Element rpint at 0x2815003d840>],
  'importance': 'minor'},
 {'message': "\npour un générateur de type {87: 'Chaudière gaz classique 1986-1990'}\nRpn < Rpint. Pour ce type de générateur c'est en général le contraire qui est attendu. \nRpn :  0.3\nRpint :  0.81\n",
  'thematique': 'incoherence_systeme',
  'objets_concerne': [<Eleme

In [19]:
el_type_generateur_c

<Element enum_type_generateur_ch_id at 0x1a53488b6c0>

In [12]:
report.error_software

[]

In [27]:
list(el_inst_ch.iterfind('*//emetteur_chauffage//enum_lien_generateur_emetteur_id'))

[<Element enum_lien_generateur_emetteur_id at 0x2b1a257eec0>]

In [18]:
el_inst_ch

<Element installation_chauffage at 0x2b1a27e0ac0>

In [29]:
lien_em_id

[1]

In [20]:
list(lien_gen_id)

[1]

In [12]:
if set([int(el) ])

In [15]:
a_list=[0,1]

In [16]:
a_list

0

In [17]:
a_list

[1]

In [9]:
report.error_software

[{'message': "\nle DPE ne correspond au modèle de DPE saisi {3: 'DPE NEUF logement  : RE2020'}\nil correspond à un des autre modèle de DPE proposé :\n{1: 'DPE 3CL 2020 méthode logement', 2: 'DPE NEUF logement : RT2012', 4: 'DPE 2006  tertiaire et ERP '}\nil manque l'objet suivant /logement_neuf/caracteristique_generale\n        ",
  'thematique': 'bad_model',
  'objets_concerne': [<lxml.etree._ElementTree at 0x1cf7bc43d00>],
  'importance': 'blocker'},
 {'message': "\nle DPE ne correspond au modèle de DPE saisi {3: 'DPE NEUF logement  : RE2020'}\nil correspond à un des autre modèle de DPE proposé :\n{1: 'DPE 3CL 2020 méthode logement', 2: 'DPE NEUF logement : RT2012', 4: 'DPE 2006  tertiaire et ERP '}\nil manque l'objet suivant /logement_neuf/sortie\n        ",
  'thematique': 'bad_model',
  'objets_concerne': [<lxml.etree._ElementTree at 0x1cf7bc43d00>],
  'importance': 'blocker'}]

In [59]:
list(el_b_0)

[]

In [68]:
self.valeur_table_dict['tv_coef_reduction_deperdition_id'].keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

In [83]:
import random

16

In [56]:
el_b_0[0].text

'0'

In [ ]:
ets = SubElement(ets_collection,'ets')
de = SubElement(ets,'donnee_entree')

In [19]:
dpe.find('*//b').text='0'

aue=dpe.find('*//surface_aue')
aue.text='0.01'
aue.getparent().getparent().find('*//b').text='0'

In [46]:
# controle que l'epaisseur du mur est déclarée lorsque une structure de mur est identifiée. 
mur_connu=dpe.xpath('*//enum_materiaux_structure_mur_id[number(text())>1]')

for el in mur_connu:
    paroi = el.getparent().getparent()
    paroi_name = paroi.tag
    de = paroi.find('donnee_entree')
    epaisseur_structure = de.find('epaisseur_structure')
    if epaisseur_structure is not None:
        msg =f"""
le type de mur {self.display_enum_traduction('enum_materiaux_structure_mur_id',int(mur_connu.text))} est connu
mais son épaisseur n'a pas été saisie.
    """
        report.generate_msg(msg,
                           msg_type='warning_saisie',
                            msg_theme='incoherence_enveloppe',
                           related_objects=[el,paroi],
                            msg_importance="critical"
                           )


In [28]:
el_aue

In [25]:
print(msg)


la valeur de b pour une paroi de type mur vaut 0
cette valeur n'est acceptable que si la surface_aue de la paroi vaut 0
        


In [13]:
report =Report()

engine.controle_coherence_table_valeur_enum(dpe,report)
print(report.error_software[1]['message'])


mauvaise correspondance entre la valeur tv_coef_reduction_deperdition_id:265
avec les données connexes suivantes :
{'enum_type_adjacence_id': [10], 'type_adjacence': 'espace tampon solarisé (véranda,loggia fermée)', 'zone_climatique': 'H1', 'enum_cfg_isolation_lnc_id': [6], 'cfg_isolation_lnc': 'lc isolé + espace tampon solarisé orienté nord', 'b': 0.95}
et l'énumérateur enum_cfg_isolation_lnc_id:{1: 'local chauffé non accessible'}.
La valeur attendue de l'énumérateur enum_cfg_isolation_lnc_id doit être une des suivantes:
{6: 'lc isolé + espace tampon solarisé orienté nord'}



In [7]:
engine.controle_coherence_tv_values_simple(dpe,report)

In [ ]:
engine.

In [10]:
ets.getchildren()

[<Element tv_coef_reduction_deperdition_id at 0x26d37816980>,
 <Element enum_cfg_isolation_lnc_id at 0x26d37816800>]

In [12]:
ets_collection.getchildren()

[]

In [39]:
el_orientation_baie_non_mur

[<Element enum_orientation_id at 0x1a7619b74c0>,
 <Element enum_orientation_id at 0x1a7619b7640>]

In [40]:
orientation_baie_ids

[1, 1]

In [58]:
dpe.find('*//baie_vitree//enum_orientation_id').text='3'

In [59]:
orientation_baie_ids = [int(el.text) for el in dpe.iterfind('*//baie_vitree//enum_orientation_id')]
# exclusion des baies horizontales
orientation_baie_ids = [el for el in orientation_baie_ids if el!=5]
# murs exterieurs sur ets ou exterieur
orientation_mur_ext= [el for el in dpe.iterfind('*//mur//enum_orientation_id') if int(el.getparent().find('enum_type_adjacence_id').text) in [1,10]]
orientation_mur_ext_id = [int(el.text) for el in orientation_mur_ext]
orientation_baie_non_mur = set(orientation_baie_ids)-set(orientation_mur_ext_id)
el_orientation_baie_non_mur = [el for el in dpe.iterfind('*//baie_vitree//enum_orientation_id') if int(el.text) in orientation_baie_non_mur]
for el_orientation in el_orientation_baie_non_mur:
    msg = f"""
une baie vitrée orientée {self.display_enum_traduction('enum_orientation_id',[int(el_orientation.text)])}
ne correspond à aucun mur donnant sur l'exterieur ou sur véranda de même orientation. 
Ceci n'est possible que dans certains cas atypiques :
- cas de paroi totalement vitrée 
- cas de velux sur un toit incliné de même orientation qu'un mur sur local non chauffé
- cas d'une paroi vitrée donnant sur un local non chauffé. 
"""
    report.generate_msg(msg, msg_type='warning_saisie',
                        msg_theme='incoherence_enveloppe',
                        related_objects=[el_orientation],
                        msg_importance='minor')


In [111]:
de=dpe.find('*//mur/donnee_entree')
epaisseur_isolation=SubElement(de,'epaisseur_isolation')
epaisseur_isolation.text='1'

In [112]:
for el in dpe.iterfind('*//epaisseur_isolation'):
    
    if int(el.text)>50:
        type_paroi = el.getparent().getparent().tag
        msg = f"""
la valeur d'epaisseur d'isolant pour une paroi de type {type_paroi} est anormalement élevée {int(el.text)} cm
vérifiez que l'epaisseur a bien été fournie en cm
    """
        report.generate_msg(msg, msg_type='warning_saisie',
                            msg_theme='incoherence_enveloppe',
                            related_objects=[el],
                            msg_importance='major')
    elif int(el.text)<2:
        type_paroi = el.getparent().getparent().tag
        msg = f"""
la valeur d'epaisseur d'isolant pour une paroi de type {type_paroi} est anormalement faible {int(el.text)} cm
vérifiez que l'epaisseur a bien été fournie en cm
    """
        report.generate_msg(msg, msg_type='warning_saisie',
                            msg_theme='incoherence_enveloppe',
                            related_objects=[el],
                            msg_importance='major')

for el in dpe.iterfind('*//epaisseur_structure'):
    
    if int(el.text)>100:
        type_paroi = el.getparent().getparent().tag
        msg = f"""
la valeur d'epaisseur de la structure pour une paroi de type {type_paroi} est anormalement élevée {int(el.text)} cm
vérifiez que l'epaisseur a bien été fournie en cm
    """
        report.generate_msg(msg, msg_type='warning_saisie',
                            msg_theme='incoherence_enveloppe',
                            related_objects=[el],
                            msg_importance='minor')
    elif int(el.text)<8:
        type_paroi = el.getparent().getparent().tag
        msg = f"""
la valeur d'epaisseur de la structure pour une paroi de type {type_paroi} est anormalement faible {int(el.text)} cm
vérifiez que l'epaisseur a bien été fournie en cm
    """
        report.generate_msg(msg, msg_type='warning_saisie',
                            msg_theme='incoherence_enveloppe',
                            related_objects=[el],
                            msg_importance='major')

In [121]:
report.warning_input

[{'message': "\nla valeur d'epaisseur d'isolant pour une paroi de type mur est anormalement élevée 60 cm\nvérifiez que l'epaisseur a bien été fournie en cm\n    ",
  'thematique': 'incoherence_enveloppe',
  'objets_concerne': [<Element epaisseur_isolation at 0x1a7626a38c0>],
  'importance': 'major'},
 {'message': "\nla valeur d'epaisseur d'isolant pour une paroi de type mur est anormalement élevée 60 cm\nvérifiez que l'epaisseur a bien été fournie en cm\n    ",
  'thematique': 'incoherence_enveloppe',
  'objets_concerne': [<Element epaisseur_isolation at 0x1a7626a38c0>],
  'importance': 'major'},
 {'message': "\nla valeur d'epaisseur d'isolant pour une paroi de type mur est anormalement faible 1 cm\nvérifiez que l'epaisseur a bien été fournie en cm\n    ",
  'thematique': 'incoherence_enveloppe',
  'objets_concerne': [<Element epaisseur_isolation at 0x1a762c6a100>],
  'importance': 'major'},
 {'message': "\nla valeur d'epaisseur de la structure pour une paroi de type mur est anormaleme

In [109]:
el.text

'60'

In [90]:
2.5+(0.0001/0.04)

2.5025

In [98]:
1/(1/2.5+(0.15/0.04))

0.24096385542168672

In [88]:
(0.1/0.04)

2.5

In [63]:
el_orientation_baie = 

In [74]:
surface_orientation_dict=dict()
surface_orientation_dict['surface_baie_vitree']=dict()
surface_orientation_dict['surface_paroi_totale']=dict()
surface_orientation_dict['surface_paroi_opaque']=dict()

for el in dpe.iterfind('*//baie_vitree//enum_orientation_id'):
    orientation_id = int(el.text)
    if orientation_id!=5:
        surf=surface_orientation_dict['surface_baie_vitree'].get(orientation_id,0)
        surf+=float(el.getparent().find('surface_totale_baie').text)
        surface_orientation_dict['surface_baie_vitree'][orientation_id]=surf
for el in dpe.iterfind('*//mur//enum_orientation_id'):
    orientation_id = int(el.text)
    surf=surface_orientation_dict['surface_paroi_opaque'].get(orientation_id,0)
    surf+=float(el.getparent().find('surface_paroi_opaque').text)
    surface_orientation_dict['surface_paroi_opaque'][orientation_id]=surf
    surf=surface_orientation_dict['surface_paroi_totale'].get(orientation_id,0)
    surf_tot = el.getparent().find('surface_paroi_opaque')
    if surf_tot is not None:
        surf+=float(surf_tot.text)

    surface_orientation_dict['surface_paroi_totale'][orientation_id]=surf

In [76]:
df = pd.DataFrame(surface_orientation_dict)
df['surface_paroi_totale_calc']=df.surface_paroi_opaque+df.surface_baie_vitree

In [61]:
print(report.warning_input[-1][('message')])


une baie vitrée orientée {3: 'Est'}
ne correspond à aucun mur donnant sur l'exterieur ou sur véranda de même orientation. 
Ceci n'est possible que dans certains cas atypiques :
- cas de paroi totalement vitrée 
- cas de velux sur un toit incliné de même orientation qu'un mur sur local non chauffé
- cas d'une paroi vitrée donnant sur un local non chauffé. 



In [57]:
el_orientation_baie_non_mur

[]

In [55]:
len(list( dpe.iterfind('*//mur//enum_orientation_id')))

3

In [56]:
for el in dpe.iterfind('*//mur//enum_orientation_id'):
    print(int(el.getparent().find('enum_type_adjacence_id').text))

1
4
17


In [ ]:
el

In [48]:
orientation_mur_ext

[]

In [47]:
set(orientation_mur_ext_id)

set()

In [46]:
aorientation_baie_non_mur

{1}

In [36]:
orientation_mur_ids

[1, 2, 3]

In [34]:
orientation_baie_ids

[1, 1]

In [18]:
dpe.find('*//enum_type_adjacence_id').text='3'

In [77]:
type_isolation_sur_lnc

[<Element enum_type_isolation_id at 0x28ff2825080>]

In [76]:
type_isolation

[<Element enum_type_isolation_id at 0x28ff2661a00>,
 <Element enum_type_isolation_id at 0x28fef885f80>,
 <Element enum_type_isolation_id at 0x28ff2825080>]

In [74]:
type_isolation

[<Element enum_type_isolation_id at 0x28ff2661a00>,
 <Element enum_type_isolation_id at 0x28fef885f80>,
 <Element enum_type_isolation_id at 0x28ff2825080>]